In [ ]:
# 일반화된 reload 함수: pyaedt_module.core의 모든 모듈을 자동으로 reload
import sys

def reload_pyaedt_modules():
    """pyaedt_module.core 패키지의 모든 모듈을 reload하고 클래스를 재import"""
    # core 패키지의 주요 모듈들 (의존성 순서대로)
    modules_to_reload = [
        'pyaedt_module.core.pydesktop',
        'pyaedt_module.core.pyproject',
        'pyaedt_module.core.pydesign',  # 선택적
        'pyaedt_module.core',
    ]
    
    # 각 모듈을 import하고 reload
    for module_name in modules_to_reload:
        if module_name in sys.modules:
            try:
                importlib.reload(sys.modules[module_name])
                print(f"✓ Reloaded: {module_name}")
            except SyntaxError as e:
                print(f"✗ Syntax error in {module_name}: {e}")
                print(f"  파일을 확인하고 문법 오류를 수정하세요!")
            except Exception as e:
                print(f"✗ Failed to reload {module_name}: {e}")
        else:
            try:
                __import__(module_name)
                importlib.reload(sys.modules[module_name])
                print(f"✓ Imported and reloaded: {module_name}")
            except SyntaxError as e:
                print(f"✗ Syntax error in {module_name}: {e}")
                print(f"  파일을 확인하고 문법 오류를 수정하세요!")
            except Exception as e:
                print(f"✗ Failed to import {module_name}: {e}")
    
    # reload 후 클래스들을 다시 import (pyDesign은 선택적)
    from pyaedt_module.core import pyDesktop, pyProject
    print("✓ Re-imported pyDesktop, pyProject")
    
    # pyDesign과 DesignList도 import
    pyDesign = None
    DesignList = None
    try:
        from pyaedt_module.core.pydesign import pyDesign, DesignList
        print("✓ Re-imported pyDesign, DesignList")
    except ImportError:
        print("⚠ pyDesign을 import할 수 없습니다 (선택적 모듈)")
    except Exception as e:
        print(f"⚠ pyDesign import 실패: {e}")
    
    return pyDesktop, pyProject, pyDesign, DesignList

def update_instance_classes(obj, pyDesktop, pyProject, pyDesign, DesignList, visited=None):
    """재귀적으로 객체의 클래스를 업데이트"""
    if visited is None:
        visited = set()
    
    # 순환 참조 방지
    obj_id = id(obj)
    if obj_id in visited:
        return
    visited.add(obj_id)
    
    try:
        # desktop 객체 업데이트
        if isinstance(obj, pyDesktop):
            obj.__class__ = pyDesktop
            # desktop.projects의 각 project 업데이트
            if hasattr(obj, 'projects'):
                try:
                    projects = obj.projects
                    for project in projects:
                        if isinstance(project, pyProject):
                            update_instance_classes(project, pyDesktop, pyProject, pyDesign, DesignList, visited)
                except Exception as e:
                    print(f"  Warning: Failed to update projects: {e}")
        
        # project 객체 업데이트
        elif isinstance(obj, pyProject):
            obj.__class__ = pyProject
            # project.designs의 각 design 업데이트
            if hasattr(obj, 'designs'):
                try:
                    designs = obj.designs
                    if isinstance(designs, DesignList):
                        designs.__class__ = DesignList
                    for design in designs:
                        if isinstance(design, pyDesign):
                            update_instance_classes(design, pyDesktop, pyProject, pyDesign, DesignList, visited)
                except Exception as e:
                    print(f"  Warning: Failed to update designs: {e}")
        
        # design 객체 업데이트
        elif pyDesign and isinstance(obj, pyDesign):
            obj.__class__ = pyDesign
        
        # Simulation 객체의 속성들도 업데이트
        if hasattr(obj, 'desktop'):
            update_instance_classes(obj.desktop, pyDesktop, pyProject, pyDesign, DesignList, visited)
        if hasattr(obj, 'project'):
            update_instance_classes(obj.project, pyDesktop, pyProject, pyDesign, DesignList, visited)
        if hasattr(obj, 'design'):
            update_instance_classes(obj.design, pyDesktop, pyProject, pyDesign, DesignList, visited)
            
    except Exception as e:
        print(f"  Warning: Failed to update {type(obj).__name__}: {e}")

# 모듈 reload 실행
pyDesktop, pyProject, pyDesign, DesignList = reload_pyaedt_modules()

# simulation_script도 reload
importlib.reload(simulation_script)

# 기존 인스턴스의 클래스를 새로 로드된 클래스로 교체
sim1.__class__ = simulation_script.Simulation

# 재귀적으로 모든 인스턴스의 클래스 업데이트
update_instance_classes(sim1, pyDesktop, pyProject, pyDesign, DesignList)

print("✓ All classes updated!")

In [1]:
import subprocess
import sys
import os

run_py = "run.py"
if not os.path.isabs(run_py):
    run_py = os.path.join(os.getcwd(), run_py)
if not os.path.isfile(run_py):
    raise FileNotFoundError(f"Cannot find 'run.py' at: {run_py}")

# python 인터프리터를 이용해 run.py 실행
result = subprocess.run([sys.executable, run_py], capture_output=True, text=True)

print(f"run.py stdout:\n{result.stdout}")
print(f"run.py stderr:\n{result.stderr}")
print(f"run.py return code: {result.returncode}")

if result.returncode != 0:
    raise RuntimeError(f"run.py failed (returncode={result.returncode})")


run.py stdout:

run.py stderr:
Traceback (most recent call last):
  File "y:\git\insulation_amp_2026\run.py", line 15, in <module>
    log_file = open(f'./simul_log/process_{i}.log', 'w')
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: './simul_log/process_0.log'

run.py return code: 1


RuntimeError: run.py failed (returncode=1)

In [2]:
import simulation_script  # 클래스를 별도 파일로 분리

sim = simulation_script.Simulation()
sim.run()

PyAEDT INFO: Python version 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.24.1.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\peets\AppData\Local\Temp\pyaedt_peets_3f383a2c-6d09-46fb-9ac7-1c3a6ed80247.log is enabled.
PyAEDT INFO: Log on AEDT is disabled.
PyAEDT INFO: Starting new AEDT gRPC session.
PyAEDT INFO: AEDT installation Path C:\Program Files\ANSYS Inc\v252\AnsysEM
PyAEDT INFO: Client application successfully started.
PyAEDT INFO: New AEDT gRPC session session started on port 50051.
PyAEDT INFO: 2025.2 version started with process ID 36744.
PyAEDT WARNING: Service Pack is not detected. PyAEDT is currently connecting in Insecure Mode.
PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Pyt

<frozen importlib._bootstrap>:241: UserWarning: The PyVista module is required to run functionalities of ansys.aedt.core.visualization.advanced.misc.
Install with 

pip install pyvista


PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Connection Correctly created
PyAEDT INFO: Boundary AutoIdentify Tx_port has been created.
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshe

GrpcApiError: Failed to execute gRPC AEDT command: GetPath

In [1]:
import importlib
import simulation_script  # 클래스를 별도 파일로 분리
import time

# 디버깅 중에는 이렇게 사용
importlib.reload(simulation_script)
sim1 = simulation_script.Simulation()


project1 = sim1.desktop.create_project(path=f"./simulation/{sim1.PROJECT_NAME}", name=sim1.PROJECT_NAME)
design1 = project1.create_design(name="HFSS_design", solver="HFSS", solution=None)


input_data = sim1.set_variable(design1)



coil_variable = {
    "color": [255, 10, 10],
    "turns": int(design1.variables["Tx_turns"]),
    "layer": int(design1.variables["Tx_layer"]),
    "outer_x": "Tx_outer_x",
    "outer_y": "Tx_outer_y",
    "fillet": "Tx_fillet",
    "inner": "Tx_inner",
    "fill_factor": "Tx_fill_factor",
    "theta1": "Tx_theta1",
    "theta2": "Tx_theta2",
    "PCB_thickness": "PCB_thickness",
    "coil_gap": "Tx_Tx_gap",
    "move": "(PCB_thickness + Tx_Rx_gap)/2",
}

Tx_winding, Tx_ter1, Tx_ter2, Tx_ter_face, Tx_width = sim1.create_winding(design1, name="Tx", up=True, **coil_variable)
design1.modeler.mirror(assignment=[Tx_winding, Tx_ter1, Tx_ter2, Tx_ter_face], origin=[0,0,0], vector=[-1,0,0])
Tx_port = sim1.create_port(design=design1, name="Tx", ter_ref=Tx_ter1, ter_face=Tx_ter_face)


coil_variable = {
    "color": [10, 10, 255],
    "turns": int(design1.variables["Rx_turns"]),
    "layer": int(design1.variables["Rx_layer"]),
    "outer_x": "Rx_outer_x",
    "outer_y": "Rx_outer_y",
    "fillet": "Rx_fillet",
    "inner": "Rx_inner",
    "fill_factor": "Rx_fill_factor",
    "theta1": "Rx_theta1",
    "theta2": "Rx_theta2",
    "PCB_thickness": "PCB_thickness",
    "coil_gap": "Rx_Rx_gap",
    "move": "-((PCB_thickness + Tx_Rx_gap)/2)",
}

Rx_winding, Rx_ter1, Rx_ter2, Rx_ter_face, Rx_width = sim1.create_winding(design1, name="Rx", up=False, **coil_variable)
Rx_port = sim1.create_port(design=design1, name="Rx", ter_ref=Rx_ter1, ter_face=Rx_ter_face)

PCB = sim1.create_PCB(design1)

design1.modeler.subtract(blank_list=PCB, tool_list=[Tx_winding, Tx_ter1, Tx_ter2, Rx_winding, Rx_ter1, Rx_ter2], keep_originals=True)

region = sim1.create_region(design1)

start_time = time.time()

setup = sim1.HFSS_analyze(project1, design1)

HFSS_results = sim1.get_HFSS_results(project1, design1)

simulation_report = sim1.simulation_report(design1, start_time)

design2 = project1.create_design(name="circuit_design", solver="Circuit", solution=None)


PyAEDT INFO: Python version 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.24.1.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\peets\AppData\Local\Temp\pyaedt_peets_74c0c42d-714d-4eca-aa83-e365102b0ab0.log is enabled.
PyAEDT INFO: Log on AEDT is disabled.
PyAEDT INFO: Starting new AEDT gRPC session.
PyAEDT INFO: AEDT installation Path C:\Program Files\ANSYS Inc\v252\AnsysEM
PyAEDT INFO: Client application successfully started.
PyAEDT INFO: New AEDT gRPC session session started on port 50051.
PyAEDT INFO: 2025.2 version started with process ID 43424.
PyAEDT WARNING: Service Pack is not detected. PyAEDT is currently connecting in Insecure Mode.
PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Pyt

<frozen importlib._bootstrap>:241: UserWarning: The PyVista module is required to run functionalities of ansys.aedt.core.visualization.advanced.misc.
Install with 

pip install pyvista


PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Connection Correctly created
PyAEDT INFO: Boundary AutoIdentify Tx_port has been created.
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Union of 3 objects has been executed.
PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshe

In [2]:
sim1.create_HFSS_link_model(link_name="HFSS_design1", project=project1, HFSS_design=design1, circuit_design=design2, Tx_port=Tx_port, Rx_port=Rx_port)

sim1.simulation_setup(circuit_design=design2)

sim1.create_schematic(circuit_design=design2)


sim1.create_output_variables(circuit_design=design2)


sim1.change_R(circuit_design=design2, R=28)
design2.Analyze("LinearFrequency")
circuit_data1 = sim1.create_report(project=project1, circuit_design=design2, name="circuit_design")

sim1.change_R(circuit_design=design2, R=50)
design2.Analyze("LinearFrequency")
circuit_data2 = sim1.create_report(project=project1, circuit_design=design2, name="circuit_design")

sim1.change_R(circuit_design=design2, R=100)
design2.Analyze("LinearFrequency")
circuit_data3 = sim1.create_report(project=project1, circuit_design=design2, name="circuit_design")

sim1.change_R(circuit_design=design2, R=200)
design2.Analyze("LinearFrequency")
circuit_data4 = sim1.create_report(project=project1, circuit_design=design2, name="circuit_design")

sim1.change_R(circuit_design=design2, R=1000)
design2.Analyze("LinearFrequency")
circuit_data5 = sim1.create_report(project=project1, circuit_design=design2, name="circuit_design")





PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


In [3]:
import pandas as pd

# 컬럼 잘림 방지 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

output_data = pd.concat([input_data, HFSS_results, circuit_data1, circuit_data2, circuit_data3, circuit_data4, circuit_data5, simulation_report], axis=1)

output_data

,Tx_turns,Rx_turns,Tx_layer,Rx_layer,Tx_outer_x,Tx_outer_y,Tx_ratio,Rx_outer_x,Rx_outer_y,Rx_ratio,Tx_inner,Rx_inner,Tx_fillet,Rx_fillet,Tx_fill_factor,Rx_fill_factor,PCB_thickness,Tx_Tx_gap,Rx_Rx_gap,Tx_Rx_gap,TT_resonant_freq,TT_resonant_Z,RR_resonant_freq,RR_resonant_Z,TR_resonant_freq,TR_resonant_Z,k_10k,k_100k,k_1M,k_10M,k_15M,k_20M,k_25M,k_30M,k_100M,LT_10k,LT_100k,LT_1M,LT_10M,LT_15M,LT_20M,LT_25M,LT_30M,LT_100M,LR_10k,LR_100k,LR_1M,LR_10M,LR_15M,LR_20M,LR_25M,LR_30M,LR_100M,LM_10k,LM_100k,LM_1M,LM_10M,LM_15M,LM_20M,LM_25M,LM_30M,LM_100M,S11_10k,S11_100k,S11_1M,S11_10M,S11_15M,S11_20M,S11_25M,S11_30M,S11_100M,S22_10k,S22_100k,S22_1M,S22_10M,S22_15M,S22_20M,S22_25M,S22_30M,S22_100M,S12_10k,S12_100k,S12_1M,S12_10M,S12_15M,S12_20M,S12_25M,S12_30M,S12_100M,S11_peak_freq,S11_peak_dB,S22_peak_freq,S22_peak_dB,S12_peak_freq,S12_peak_dB,Pincircuit_design,Poutcircuit_design,Gvcircuit_design,effcircuit_design,Vincircuit_design,Vloadcircuit_design,Iincircuit_design,Iloadcircuit_design,Pincircuit_design,Poutcircuit_design,Gvcircuit_design,effcircuit_design,Vincircuit_design,Vloadcircuit_design,Iincircuit_design,Iloadcircuit_design,Pincircuit_design,Poutcircuit_design,Gvcircuit_design,effcircuit_design,Vincircuit_design,Vloadcircuit_design,Iincircuit_design,Iloadcircuit_design,Pincircuit_design,Poutcircuit_design,Gvcircuit_design,effcircuit_design,Vincircuit_design,Vloadcircuit_design,Iincircuit_design,Iloadcircuit_design,Pincircuit_design,Poutcircuit_design,Gvcircuit_design,effcircuit_design,Vincircuit_design,Vloadcircuit_design,Iincircuit_design,Iloadcircuit_design,time,pass_number,tetrahedra,delta_S
0,8,13,1,2,1.72,1.8,1.046512,2.03,1.49,0.73399,0.97,0.94,0.82,0.82,0.57,0.59,0.0126,0.053,0.038,0.185,136.491228,1717.738,136.491228,9433.046,136.491228,4019.297,0.150881,0.337471,0.56656,0.656503,0.661906,0.666125,0.670052,0.673378,0.79204,430.06408,418.589884,232.390049,206.861244,206.262806,206.050066,206.130909,206.221161,295.994512,561.485375,561.067962,554.888435,501.844987,503.574357,507.87234,512.823942,516.193336,1014.214776,-74.142956,-163.545306,-203.449862,-211.524839,-213.323319,-215.486247,-217.853365,-219.700891,-433.963946,0.223136,0.23211,0.396215,1.352341,1.905696,2.295921,2.535012,2.658282,1.472179,0.027522,0.027956,0.08425,1.307691,1.794991,2.087552,2.221067,2.236158,1.217845,45.847297,44.388576,26.041886,8.059411,6.250346,5.407787,4.995357,4.795468,6.915802,34.561404,2.698113,448.421053,3.946307,34.561404,4.613713,0.351,0.208,0.7786,59.2768,4.3841,3.4133,0.399,0.1219,0.2639,0.1542,0.8957,58.428,4.3841,3.9269,0.3592,0.0785,0.1835,0.0894,0.9643,48.6986,4.3841,4.2277,0.3395,0.0423,0.1375,0.0473,0.9927,34.4318,4.3841,4.352,0.3347,0.0218,0.0993,0.0098,1.011,9.8894,4.3841,4.4323,0.3343,0.0044,123,6,25473,0.00072575


In [5]:
import os

current_dir = os.getcwd()
csv_file = os.path.join(current_dir, f"output_data_circuit.csv")

if os.path.isfile(csv_file):
    output_data.to_csv(csv_file, mode='a', index=False, header=False)
else:
    output_data.to_csv(csv_file, mode='w', index=False, header=True)


    

In [ ]:
project1.close()
time.sleep(1)
project1.delete_project_folder(path=project1.path)

In [6]:
Tx_terminal_name = list(Tx_port.children.keys())[0]
Rx_terminal_name = list(Rx_port.children.keys())[0]

Tx_terminal_name

'Tx_port_T1'

In [11]:
import os

link_name = "HFSS_design2"
project = project1
HFSS_design = design1
circuit_design = design2
Tx_port = Tx_port
Rx_port = Rx_port

image_file = os.path.join(project.path, "image.gif")
ModTime = int(time.time())

oDefinitionManager = project.GetDefinitionManager()
oModelManager = oDefinitionManager.GetManager("Model")
oComponentManager = oDefinitionManager.GetManager("Component")

Tx_terminal_name = list(Tx_port.children.keys())[0]
Rx_terminal_name = list(Rx_port.children.keys())[0]

# oModelManager로 추가
params1 = [
f"NAME:{link_name}",
"Name:="		, f"{link_name}",
"ModTime:="		, f"{ModTime}",
"Library:="		, "",
"LibLocation:="		, "Project",
"ModelType:="		, "hfss",
"Description:="		, "",
"ImageFile:="		, f"{image_file}",
"SymbolPinConfiguration:=", 0,
[
    "NAME:PortInfoBlk"
],
[
    "NAME:PortOrderBlk"
],
"DesignName:="		, f"{HFSS_design.name}",
"SolutionName:="	, "Setup1 : Sweep",
"NewToOldMap:="		, [],
"OldToNewMap:="		, [],
"PinNames:="		, [f"{Rx_terminal_name}",f"{Tx_terminal_name}"],
[
    "NAME:DesignerCustomization",
    "DCOption:="		, 0,
    "InterpOption:="	, 0,
    "ExtrapOption:="	, 1,
    "Convolution:="		, 0,
    "Passivity:="		, 0,
    "Reciprocal:="		, False,
    "ModelOption:="		, "",
    "DataType:="		, 1
],
[
    "NAME:NexximCustomization",
    "DCOption:="		, 3,
    "InterpOption:="	, 1,
    "ExtrapOption:="	, 3,
    "Convolution:="		, 0,
    "Passivity:="		, 0,
    "Reciprocal:="		, False,
    "ModelOption:="		, "",
    "DataType:="		, 2
],
[
    "NAME:HSpiceCustomization",
    "DCOption:="		, 1,
    "InterpOption:="	, 2,
    "ExtrapOption:="	, 3,
    "Convolution:="		, 0,
    "Passivity:="		, 0,
    "Reciprocal:="		, False,
    "ModelOption:="		, "",
    "DataType:="		, 3
],
"NoiseModelOption:="	, "External",
"WB_SystemID:="		, f"{HFSS_design.name}",
"IsWBModel:="		, False,
"filename:="		, "",
"numberofports:="	, 2,
"Simulate:="		, False,
"CloseProject:="	, False,
"SaveProject:="		, True,
"InterpY:="		, True,
"InterpAlg:="		, "auto",
"IgnoreDepVars:="	, False,
"Renormalize:="		, False,
"RenormImpedance:="	, 50
]


# oComponentManager로 추가
params2 = [
f"NAME:{link_name}",
"Info:="		, [			"Type:="		, 8,			"NumTerminals:="	, 2,			"DataSource:="		, "",			"ModifiedOn:="		, ModTime,			"Manufacturer:="	, "",			"Symbol:="		, "",			"ModelNames:="		, "",			"Footprint:="		, "",			"Description:="		, "",			"InfoTopic:="		, "",			"InfoHelpFile:="	, "",			"IconFile:="		, "hfss.bmp",			"Library:="		, "",			"OriginalLocation:="	, "Project",			"IEEE:="		, "",			"Author:="		, "",			"OriginalAuthor:="	, "",			"CreationDate:="	, 1769047810,			"ExampleFile:="		, "",			"HiddenComponent:="	, 0,			"CircuitEnv:="		, 0,			"GroupID:="		, 0],
"CircuitEnv:="		, 0,
"Refbase:="		, "S",
"NumParts:="		, 1,
"ModSinceLib:="		, False,
"Terminal:="		, [f"{Rx_terminal_name}",f"{Rx_terminal_name}","A",False,2,1,"","Electrical","0"],
"Terminal:="		, [f"{Tx_terminal_name}",f"{Tx_terminal_name}","A",False,3,1,"","Electrical","0"],
[
    "NAME:Properties",
    "TextProp:="		, ["Owner","RD","","HFSS"]
],
"CompExtID:="		, 5,
[
    "NAME:Parameters",
    "VariableProp:="	, ["Tx_outer_x","D","",f"{HFSS_design.variables['Tx_outer_x']}"],
    "VariableProp:="	, ["Rx_outer_x","D","",f"{HFSS_design.variables['Rx_outer_x']}"],
    "VariableProp:="	, ["Tx_inner","D","",f"{HFSS_design.variables['Tx_inner']}"],
    "VariableProp:="	, ["Tx_fillet","D","",f"{HFSS_design.variables['Tx_fillet']}"],
    "VariableProp:="	, ["Tx_fill_factor","D","",f"{HFSS_design.variables['Tx_fill_factor']}"],
    "VariableProp:="	, ["Rx_inner","D","",f"{HFSS_design.variables['Rx_inner']}"],
    "VariableProp:="	, ["Rx_fillet","D","",f"{HFSS_design.variables['Rx_fillet']}"],
    "VariableProp:="	, ["Rx_fill_factor","D","",f"{HFSS_design.variables['Rx_fill_factor']}"],
    "VariableProp:="	, ["Tx_turns","D","",f"{HFSS_design.variables['Tx_turns']}"],
    "VariableProp:="	, ["Rx_turns","D","",f"{HFSS_design.variables['Rx_turns']}"],
    "VariableProp:="	, ["Tx_layer","D","",f"{HFSS_design.variables['Tx_layer']}"],
    "VariableProp:="	, ["Rx_layer","D","",f"{HFSS_design.variables['Rx_layer']}"],
    "VariableProp:="	, ["Tx_outer_y","HD","",f"{HFSS_design.variables['Tx_outer_y']}"],
    "VariableProp:="	, ["Rx_outer_y","HD","",f"{HFSS_design.variables['Rx_outer_y']}"],
    "VariableProp:="	, ["Tx_theta1","HD","",f"{HFSS_design.variables['Tx_theta1']}"],
    "VariableProp:="	, ["Tx_theta2","HD","",f"{HFSS_design.variables['Tx_theta2']}"],
    "VariableProp:="	, ["Rx_theta1","HD","",f"{HFSS_design.variables['Rx_theta1']}"],
    "VariableProp:="	, ["Rx_theta2","HD","",f"{HFSS_design.variables['Rx_theta2']}"],
    "VariableProp:="	, ["PCB_thickness","D","",f"{HFSS_design.variables['PCB_thickness']}"],
    "VariableProp:="	, ["Tx_Tx_gap","D","",f"{HFSS_design.variables['Tx_Tx_gap']}"],
    "VariableProp:="	, ["Rx_Rx_gap","D","",f"{HFSS_design.variables['Rx_Rx_gap']}"],
    "VariableProp:="	, ["Tx_Rx_gap","D","",f"{HFSS_design.variables['Tx_Rx_gap']}"],
    "TextProp:="		, ["ModelName","RD","","FieldSolver"],
    "MenuProp:="		, ["CoSimulator","SD","","Default",0],
    "ButtonProp:="		, ["CosimDefinition","SD","","Edit","Edit",40501,				"ButtonPropClientData:=", []]
],
[
    "NAME:CosimDefinitions",
    [
        "NAME:CosimDefinition",
        "CosimulatorType:="	, 103,
        "CosimDefName:="	, "Default",
        "IsDefinition:="	, True,
        "Connect:="		, True,
        "ModelDefinitionName:="	, f"{link_name}",
        "ShowRefPin2:="		, 2,
        "LenPropName:="		, ""
    ],
    "DefaultCosim:="	, "Default"
]
]

oModelManager.Add(params1)
oComponentManager.Add(params2)
circuit_design.AddCompInstance(f"{link_name}")



In [3]:
import os
import time

HFSS_design = design1
circuit_design = design2

Tx_port = Tx_port
Rx_port = Rx_port

link_name = "HFSS_link_model"


image_file = os.path.join(project1.path, "image.gif")
ModTime = int(time.time())

oDefinitionManager = project1.GetDefinitionManager()
oModelManager = oDefinitionManager.GetManager("Model")
oComponentManager = oDefinitionManager.GetManager("Component")

Tx_terminal_name = list(Tx_port.children.keys())[0]
Rx_terminal_name = list(Rx_port.children.keys())[0]

# oModelManager로 추가
params1 = [
    "NAME:HFSS_design1",
    "Name:="		, "HFSS_design1",
    "ModTime:="		, f"{ModTime}",
    "Library:="		, "",
    "LibLocation:="		, "Project",
    "ModelType:="		, "hfss",
    "Description:="		, "",
    "ImageFile:="		, f"{image_file}",
    "SymbolPinConfiguration:=", 0,
    [
        "NAME:PortInfoBlk"
    ],
    [
        "NAME:PortOrderBlk"
    ],
    "DesignName:="		, f"{HFSS_design.name}",
    "SolutionName:="	, "Setup1 : Sweep",
    "NewToOldMap:="		, [],
    "OldToNewMap:="		, [],
    "PinNames:="		, [f"{Rx_terminal_name}",f"{Tx_terminal_name}"],
    [
        "NAME:DesignerCustomization",
        "DCOption:="		, 0,
        "InterpOption:="	, 0,
        "ExtrapOption:="	, 1,
        "Convolution:="		, 0,
        "Passivity:="		, 0,
        "Reciprocal:="		, False,
        "ModelOption:="		, "",
        "DataType:="		, 1
    ],
    [
        "NAME:NexximCustomization",
        "DCOption:="		, 3,
        "InterpOption:="	, 1,
        "ExtrapOption:="	, 3,
        "Convolution:="		, 0,
        "Passivity:="		, 0,
        "Reciprocal:="		, False,
        "ModelOption:="		, "",
        "DataType:="		, 2
    ],
    [
        "NAME:HSpiceCustomization",
        "DCOption:="		, 1,
        "InterpOption:="	, 2,
        "ExtrapOption:="	, 3,
        "Convolution:="		, 0,
        "Passivity:="		, 0,
        "Reciprocal:="		, False,
        "ModelOption:="		, "",
        "DataType:="		, 3
    ],
    "NoiseModelOption:="	, "External",
    "WB_SystemID:="		, f"{HFSS_design.name}",
    "IsWBModel:="		, False,
    "filename:="		, "",
    "numberofports:="	, 2,
    "Simulate:="		, False,
    "CloseProject:="	, False,
    "SaveProject:="		, True,
    "InterpY:="		, True,
    "InterpAlg:="		, "auto",
    "IgnoreDepVars:="	, False,
    "Renormalize:="		, False,
    "RenormImpedance:="	, 50
]


# oComponentManager로 추가
params2 = [
    f"NAME:{link_name}",
    "Info:="		, [			"Type:="		, 8,			"NumTerminals:="	, 2,			"DataSource:="		, "",			"ModifiedOn:="		, ModTime,			"Manufacturer:="	, "",			"Symbol:="		, "",			"ModelNames:="		, "",			"Footprint:="		, "",			"Description:="		, "",			"InfoTopic:="		, "",			"InfoHelpFile:="	, "",			"IconFile:="		, "hfss.bmp",			"Library:="		, "",			"OriginalLocation:="	, "Project",			"IEEE:="		, "",			"Author:="		, "",			"OriginalAuthor:="	, "",			"CreationDate:="	, 1769047810,			"ExampleFile:="		, "",			"HiddenComponent:="	, 0,			"CircuitEnv:="		, 0,			"GroupID:="		, 0],
    "CircuitEnv:="		, 0,
    "Refbase:="		, "S",
    "NumParts:="		, 1,
    "ModSinceLib:="		, False,
    "Terminal:="		, [f"{Rx_terminal_name}",f"{Rx_terminal_name}","A",False,2,1,"","Electrical","0"],
    "Terminal:="		, [f"{Tx_terminal_name}",f"{Tx_terminal_name}","A",False,3,1,"","Electrical","0"],
    [
        "NAME:Properties",
        "TextProp:="		, ["Owner","RD","","HFSS"]
    ],
    "CompExtID:="		, 5,
    [
        "NAME:Parameters",
        "VariableProp:="	, ["Tx_outer_x","D","",f"{HFSS_design.variables['Tx_outer_x']}"],
        "VariableProp:="	, ["Rx_outer_x","D","",f"{HFSS_design.variables['Rx_outer_x']}"],
        "VariableProp:="	, ["Tx_inner","D","",f"{HFSS_design.variables['Tx_inner']}"],
        "VariableProp:="	, ["Tx_fillet","D","",f"{HFSS_design.variables['Tx_fillet']}"],
        "VariableProp:="	, ["Tx_fill_factor","D","",f"{HFSS_design.variables['Tx_fill_factor']}"],
        "VariableProp:="	, ["Rx_inner","D","",f"{HFSS_design.variables['Rx_inner']}"],
        "VariableProp:="	, ["Rx_fillet","D","",f"{HFSS_design.variables['Rx_fillet']}"],
        "VariableProp:="	, ["Rx_fill_factor","D","",f"{HFSS_design.variables['Rx_fill_factor']}"],
        "VariableProp:="	, ["Tx_turns","D","",f"{HFSS_design.variables['Tx_turns']}"],
        "VariableProp:="	, ["Rx_turns","D","",f"{HFSS_design.variables['Rx_turns']}"],
        "VariableProp:="	, ["Tx_layer","D","",f"{HFSS_design.variables['Tx_layer']}"],
        "VariableProp:="	, ["Rx_layer","D","",f"{HFSS_design.variables['Rx_layer']}"],
        "VariableProp:="	, ["Tx_outer_y","HD","",f"{HFSS_design.variables['Tx_outer_y']}"],
        "VariableProp:="	, ["Rx_outer_y","HD","",f"{HFSS_design.variables['Rx_outer_y']}"],
        "VariableProp:="	, ["Tx_theta1","HD","",f"{HFSS_design.variables['Tx_theta1']}"],
        "VariableProp:="	, ["Tx_theta2","HD","",f"{HFSS_design.variables['Tx_theta2']}"],
        "VariableProp:="	, ["Rx_theta1","HD","",f"{HFSS_design.variables['Rx_theta1']}"],
        "VariableProp:="	, ["Rx_theta2","HD","",f"{HFSS_design.variables['Rx_theta2']}"],
        "VariableProp:="	, ["PCB_thickness","D","",f"{HFSS_design.variables['PCB_thickness']}"],
        "VariableProp:="	, ["Tx_Tx_gap","D","",f"{HFSS_design.variables['Tx_Tx_gap']}"],
        "VariableProp:="	, ["Rx_Rx_gap","D","",f"{HFSS_design.variables['Rx_Rx_gap']}"],
        "VariableProp:="	, ["Tx_Rx_gap","D","",f"{HFSS_design.variables['Tx_Rx_gap']}"],
        "TextProp:="		, ["ModelName","RD","","FieldSolver"],
        "MenuProp:="		, ["CoSimulator","SD","","Default",0],
        "ButtonProp:="		, ["CosimDefinition","SD","","Edit","Edit",40501,				"ButtonPropClientData:=", []]
    ],
    [
        "NAME:CosimDefinitions",
        [
            "NAME:CosimDefinition",
            "CosimulatorType:="	, 103,
            "CosimDefName:="	, "Default",
            "IsDefinition:="	, True,
            "Connect:="		, True,
            "ModelDefinitionName:="	, f"{link_name}",
            "ShowRefPin2:="		, 2,
            "LenPropName:="		, ""
        ],
        "DefaultCosim:="	, "Default"
    ]
]




In [3]:
oModelManager.Add(params1)
circuit_design.AddCompInstance(f"{link_name}")


In [4]:

oModelManager.Add(params1)
oComponentManager.Add(params2)


circuit_design.AddCompInstance(f"{link_name}")



In [26]:
dir(link)

['__bool__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [9]:
oModule = circuit_design.GetModule("SimSetup")
oModule.AddLinearNetworkAnalysis(
    [
        "NAME:SimSetup",
        "DataBlockID:="		, 16,
        "OptionName:="		, "(Default Options)",
        "AdditionalOptions:="	, "",
        "AlterBlockName:="	, "",
        "FilterText:="		, "",
        "AnalysisEnabled:="	, 1,
        "HasTDRComp:="		, 0,
        [
            "NAME:OutputQuantities"
        ],
        [
            "NAME:NoiseOutputQuantities"
        ],
        "Name:="		, "LinearFrequency",
        "LinearFrequencyData:="	, [False,0.1,False,"",False],
        [
            "NAME:SweepDefinition",
            "Variable:="		, "Freq",
            "Data:="		, "10MHz",
            "OffsetF1:="		, False,
            "Synchronize:="		, 0
        ]
    ])

In [11]:
oDesign = circuit_design
oEditor = oDesign.SetActiveEditor("SchematicEditor")

oEditor.CreateIPort(
    [
        "NAME:IPortProps",
        "Name:="		, "Port1",
        "Id:="			, 3
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1,
        "X:="			, 0.0127,
        "Y:="			, 0.0127,
        "Angle:="		, 0,
        "Flip:="		, False
    ])
oDesign.UpdateSources(
    [
        "NAME:NexximSources",
        [
            "NAME:NexximSources",
            [
                "NAME:Data"
            ]
        ]
    ], 
    [
        "NAME:ComponentConfigurationData",
        [
            "NAME:ComponentConfigurationData",
            [
                "NAME:EnabledPorts"
            ],
            [
                "NAME:EnabledMultipleComponents"
            ],
            [
                "NAME:EnabledAnalyses",
                [
                    "NAME:Port1",
                    "Port1:="		, ["LinearFrequency"]
                ]
            ]
        ]
    ])
oDesign.ChangePortProperty("Port1", 
    [
        "NAME:Port1",
        "IIPortName:="		, "Port1",
        "SymbolType:="		, 1,
        "DoPostProcess:="	, False
    ], 
    [
        [
            "NAME:Properties",
            [
                "NAME:ChangedProps",
                [
                    "NAME:rz",
                    "Value:="		, "50ohm"
                ],
                [
                    "NAME:iz",
                    "Value:="		, "0ohm"
                ],
                [
                    "NAME:pnum",
                    "Value:="		, "1"
                ],
                [
                    "NAME:EnableNoise",
                    "Value:="		, False
                ],
                [
                    "NAME:noisetemp",
                    "Value:="		, "16.85cel"
                ]
            ]
        ]
    ])

oDesign.UpdateSources(
    [
        "NAME:NexximSources",
        [
            "NAME:NexximSources",
            [
                "NAME:Data",
                [
                    "NAME:VoltageSinusoidal1",
                    "DataId:="		, "Source0",
                    "Type:="		, 1,
                    "Output:="		, 0,
                    "NumPins:="		, 2,
                    "Netlist:="		, "V@ID %0 %1 *DC(DC=@DC) SIN(?VO(@VO) ?VA(@VA) ?FREQ(@FREQ) ?TD(@TD) ?ALPHA(@ALPHA) ?THETA(@THETA)) *TONE(TONE=@TONE) *ACMAG(AC @ACMAG @ACPHASE)",
                    "CompName:="		, "Nexxim Circuit Elements\\Independent Sources:V_SIN",
                    "FDSFileName:="		, "",
                    "BtnPropFileName:="	, "",
                    [
                        "NAME:Properties",
                        "TextProp:="		, ["LabelID","HD","Property string for netlist ID","V@ID"],
                        "ValueProp:="		, ["ACMAG","OD","AC magnitude for small-signal analysis (Volts)","3.1*sqrt(2) V",0],
                        "ValuePropNU:="		, ["ACPHASE","D","AC phase for small-signal analysis","0deg",0,"deg",							"AdditionalPropInfo:="	, ""],
                        "ValueProp:="		, ["DC","D","DC voltage (Volts)","0V",0],
                        "ValueProp:="		, ["VO","D","Voltage offset from zero (Volts)","0V",0],
                        "ValueProp:="		, ["VA","D","Voltage amplitude (Volts)","0V",0],
                        "ValueProp:="		, ["FREQ","OD","Frequency (Hz)","10MHz",0],
                        "ValueProp:="		, ["TD","D","Delay to start of sine wave (seconds)","0s",0],
                        "ValueProp:="		, ["ALPHA","D","Damping factor (1/seconds)","0",0],
                        "ValuePropNU:="		, ["THETA","D","Phase delay","0deg",0,"deg",							"AdditionalPropInfo:="	, ""],
                        "ValueProp:="		, ["TONE","D","Frequency (Hz) to use for harmonic balance analysis, should be a submultiple of (or equal to) the driving frequency and should also be included in the HB analysis setup","0Hz",0],
                        "TextProp:="		, ["ModelName","SHD","","V_SIN"],
                        "ButtonProp:="		, ["CosimDefinition","D","","Edit","Edit",40501,							"ButtonPropClientData:=", []],
                        "MenuProp:="		, ["CoSimulator","D","","DefaultNetlist",0]
                    ]
                ]
            ]
        ]
    ], 
    [
        "NAME:ComponentConfigurationData",
        [
            "NAME:ComponentConfigurationData",
            [
                "NAME:EnabledPorts",
                "VoltageSinusoidal1:="	, ["Port1"]
            ],
            [
                "NAME:EnabledMultipleComponents",
                "VoltageSinusoidal1:="	, []
            ],
            [
                "NAME:EnabledAnalyses",
                [
                    "NAME:Port1",
                    "Port1:="		, ["LinearFrequency"]
                ],
                [
                    "NAME:VoltageSinusoidal1",
                    "Port1:="		, ["LinearFrequency"]
                ]
            ]
        ]
    ])
oDesign.ChangePortProperty("Port1", 
    [
        "NAME:Port1",
        "IIPortName:="		, "Port1",
        "SymbolType:="		, 1,
        "DoPostProcess:="	, False
    ], 
    [
        [
            "NAME:Properties",
            [
                "NAME:ChangedProps",
                [
                    "NAME:rz",
                    "Value:="		, "1e-06ohm"
                ],
                [
                    "NAME:iz",
                    "Value:="		, "0ohm"
                ],
                [
                    "NAME:pnum",
                    "Value:="		, "1"
                ],
                [
                    "NAME:EnableNoise",
                    "Value:="		, False
                ],
                [
                    "NAME:noisetemp",
                    "Value:="		, "16.85cel"
                ]
            ]
        ]
    ])
oEditor.CreateComponent(
    [
        "NAME:ComponentProps",
        "Name:="		, "Nexxim Circuit Elements\\Probes:IPROBE",
        "Id:="			, "23"
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1,
        "X:="			, 0.01524,
        "Y:="			, 0.00254,
        "Angle:="		, 0,
        "Flip:="		, False
    ])
oEditor.Move(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@IPROBE;23;12"]
    ], 
    [
        "NAME:MoveParameters",
        "xdelta:="		, -0.00254,
        "ydelta:="		, 0.00254,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.Rotate(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@IPROBE;23;12"]
    ], 
    [
        "NAME:RotateParameters",
        "Degrees:="		, 90,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.Rotate(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@IPROBE;23;12"]
    ], 
    [
        "NAME:RotateParameters",
        "Degrees:="		, 90,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.Rotate(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@IPROBE;23;12"]
    ], 
    [
        "NAME:RotateParameters",
        "Degrees:="		, 90,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.ChangeProperty(
    [
        "NAME:AllTabs",
        [
            "NAME:PassedParameterTab",
            [
                "NAME:PropServers", 
                "CompInst@IPROBE;23;12:1"
            ],
            [
                "NAME:ChangedProps",
                [
                    "NAME:Name",
                    "Value:="		, "Iin"
                ]
            ]
        ]
    ])
oEditor.CreateComponent(
    [
        "NAME:ComponentProps",
        "Name:="		, "Nexxim Circuit Elements\\Probes:IPROBE",
        "Id:="			, "24"
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1,
        "X:="			, -0.03302,
        "Y:="			, -0.00254,
        "Angle:="		, 0,
        "Flip:="		, False
    ])
oEditor.Rotate(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@IPROBE;24;19"]
    ], 
    [
        "NAME:RotateParameters",
        "Degrees:="		, 90,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.Rotate(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@IPROBE;24;19"]
    ], 
    [
        "NAME:RotateParameters",
        "Degrees:="		, 90,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.ChangeProperty(
    [
        "NAME:AllTabs",
        [
            "NAME:PassedParameterTab",
            [
                "NAME:PropServers", 
                "CompInst@IPROBE;24;19:1"
            ],
            [
                "NAME:ChangedProps",
                [
                    "NAME:Name",
                    "Value:="		, "Iout"
                ]
            ]
        ]
    ])
oEditor.CreateComponent(
    [
        "NAME:ComponentProps",
        "Name:="		, "Nexxim Circuit Elements\\Resistors:RES_",
        "Id:="			, "25"
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1,
        "X:="			, -0.0508,
        "Y:="			, -0.00254,
        "Angle:="		, 0,
        "Flip:="		, False
    ])
oEditor.CreateWire(
    [
        "NAME:WireData",
        "Name:="		, "",
        "Id:="			, 26,
        "Points:="		, ["(-0.045720, -0.002540)","(-0.038100, -0.002540)"]
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1
    ])
oEditor.CreateWire(
    [
        "NAME:WireData",
        "Name:="		, "",
        "Id:="			, 32,
        "Points:="		, ["(-0.027940, -0.002540)","(-0.010160, -0.002540)"]
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1
    ])
oEditor.CreateWire(
    [
        "NAME:WireData",
        "Name:="		, "",
        "Id:="			, 38,
        "Points:="		, ["(0.000000, -0.002540)","(0.012700, -0.002540)","(0.012700, 0.000000)"]
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1
    ])
oEditor.CreateGround(
    [
        "NAME:GroundProps",
        "Id:="			, 44
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1,
        "X:="			, -0.05588,
        "Y:="			, -0.01778,
        "Angle:="		, 0,
        "Flip:="		, False
    ])
oEditor.CreateWire(
    [
        "NAME:WireData",
        "Name:="		, "",
        "Id:="			, 48,
        "Points:="		, ["(-0.055880, -0.015240)","(-0.055880, -0.002540)"]
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1
    ])
oEditor.ChangeProperty(
    [
        "NAME:AllTabs",
        [
            "NAME:PassedParameterTab",
            [
                "NAME:PropServers", 
                "CompInst@RES_;25;24:1"
            ],
            [
                "NAME:ChangedProps",
                [
                    "NAME:R",
                    "Value:="		, "28"
                ]
            ]
        ]
    ])
oEditor.CreateWire(
    [
        "NAME:WireData",
        "Name:="		, "",
        "Id:="			, 53,
        "Points:="		, ["(0.012700, 0.010160)","(0.012700, 0.012700)"]
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1
    ])
oEditor.CreateComponent(
    [
        "NAME:ComponentProps",
        "Name:="		, "Nexxim Circuit Elements\\Probes:VPROBE",
        "Id:="			, "26"
    ], 
    [
        "NAME:Attributes",
        "Page:="		, 1,
        "X:="			, 0.0254,
        "Y:="			, -0.00508,
        "Angle:="		, 0,
        "Flip:="		, False
    ])
oEditor.Move(
    [
        "NAME:Selections",
        "Selections:="		, ["CompInst@VPROBE;26;57"]
    ], 
    [
        "NAME:MoveParameters",
        "xdelta:="		, -0.06604,
        "ydelta:="		, 0.00762,
        "Disconnect:="		, False,
        "Rubberband:="		, False
    ])
oEditor.ChangeProperty(
    [
        "NAME:AllTabs",
        [
            "NAME:PassedParameterTab",
            [
                "NAME:PropServers", 
                "CompInst@VPROBE;26;57:1"
            ],
            [
                "NAME:ChangedProps",
                [
                    "NAME:Name",
                    "Value:="		, "Vout"
                ]
            ]
        ]
    ])

In [12]:
oDesign.Analyze("LinearFrequency")

In [ ]:
import pandas as pd

In [ ]:
oModule = oDesign.GetModule("OutputVariable")
oModule.CreateOutputVariable("Pin", "0.5*re(V(Port1)*conjg(I(Iin)))", "LinearFrequency", "Standard", 
    [
        "NAME:Context",
        "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
    ])
oModule.CreateOutputVariable("Pout", "0.5*re(V(Vout)*conjg(I(Iout)))", "LinearFrequency", "Standard", 
    [
        "NAME:Context",
        "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
    ])
oModule.CreateOutputVariable("Gv", "mag(V(Vout))/mag(V(Port1))", "LinearFrequency", "Standard", 
    [
        "NAME:Context",
        "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
    ])
oModule.CreateOutputVariable("Eff", "Pout/Pin*100", "LinearFrequency", "Standard", 
    [
        "NAME:Context",
        "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
    ])




GrpcApiError: Failed to execute gRPC AEDT command: CreateOutputVariable

In [16]:

oModule = oDesign.GetModule("ReportSetup")
oModule.CreateReport("Return Loss Table 1", "Standard", "Data Table", "LinearFrequency", 
    [
        "NAME:Context",
        "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
    ], 
    [
        "Freq:="		, ["All"]
    ], 
    [
        "X Component:="		, "Freq",
        "Y Component:="		, ["Pin","Pout","Gv","Eff","mag(V(Port1))","mag(V(Vout))","mag(I(Iin))","mag(I(Iout))"]
    ])



project_dir = project1.path

sim_data_RL = circuit_design.post.export_report_to_csv(project_dir=project_dir, plot_name="Return Loss Table 1", uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

data_RL = pd.read_csv(sim_data_RL)
data_RL = circuit_design.post_processing.data_preprocessing(data_RL)

# 단위 변환 및 컬럼명 변경
rename_dict = {}
for col in data_RL.columns:
    if "[mV]" in col:
        data_RL[col] = data_RL[col] / 1000  # mV → V
        new_col = col.replace("[mV]", "[V]")
        rename_dict[col] = new_col
    elif "[mA]" in col:
        data_RL[col] = data_RL[col] / 1000  # mA → A
        new_col = col.replace("[mA]", "[A]")
        rename_dict[col] = new_col

# 컬럼 이름 일괄 변경
data_RL.rename(columns=rename_dict, inplace=True)

data_Pin = round(data_RL["Pin []"].values[0],4)
data_Pout = round(data_RL["Pout []"].values[0],4)
data_Gv = round(data_RL["Gv []"].values[0],4)
data_eff = round(data_RL["Eff []"].values[0],4)
data_Vin = round(data_RL["mag(V(Port1)) [V]"].values[0],4)
data_Vload = round(data_RL["mag(V(Vout)) [V]"].values[0],4)
data_Iin = round(data_RL["mag(I(Iin)) [A]"].values[0],4)
data_Iload = round(data_RL["mag(I(Iout)) [A]"].values[0],4)

columns = ['Pin28', 'Pout28', 'Gv28', 'eff28', 'Vin28', 'Vload28', 'Iin28', 'Iload28']
circuit_data_raw = [data_Pin, data_Pout, data_Gv, data_eff, data_Vin, data_Vload, data_Iin, data_Iload]
circuit_data_28 = pd.DataFrame([circuit_data_raw], columns=columns)

PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


NameError: name 'pd' is not defined

In [ ]:
input_data

In [ ]:
import pandas as pd

# 컬럼(열)이 많으면 pandas가 자동으로 일부만 보여주기 때문에 옵션 조정
pd.set_option('display.max_columns', None)  # 모든 컬럼 출력
pd.set_option('display.width', None)        # 줄바꿈 없이 한 줄로 출력

HFSS_results

In [ ]:
simulation_report

In [ ]:
project_dir = project1.path # csv 파일 저장 위치치
design = design1

# resonant frequency / coupling coefficient data

result_expressions = []
result_expressions.append(f"mag(Zt(Tx_port_T1,Tx_port_T1))")
result_expressions.append(f"mag(Zt(Rx_port_T1,Rx_port_T1))")
result_expressions.append(f"mag(Zt(Tx_port_T1,Rx_port_T1))")
result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))/sqrt(im(Zt(Tx_port_T1,Tx_port_T1))*im(Zt(Rx_port_T1,Rx_port_T1)))")

report1 = design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                report_category="Terminal Solution Data", plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="Impedance Data")
time.sleep(1)
sim_data1 = design.post.export_report_to_csv(project_dir=project_dir, plot_name=report1.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
data1 = pd.read_csv(sim_data1)
data1 = design.post_processing.data_preprocessing(data1)

data_freq = data1["Freq [Hz]"]
data_ZTT = data1["mag(Zt(Tx_port_T1,Tx_port_T1)) []"]
data_ZRR = data1["mag(Zt(Rx_port_T1,Rx_port_T1)) []"]
data_ZTR = data1["mag(Zt(Tx_port_T1,Rx_port_T1)) []"]
data_k = data1["im(Zt(Tx_port_T1,Rx_port_T1))/sqrt(im(Zt(Tx_port_T1,Tx_port_T1))*im(Zt(Rx_port_T1,Rx_port_T1))) []"]


freq1, peak1 = design.post_processing.detect_peak(freq=data_freq, data=data_ZTT) # Z11 공진 주파수
freq2, peak2 = design.post_processing.detect_peak(freq=data_freq, data=data_ZRR) # Z22 공진주파수
freq3, peak3 = design.post_processing.detect_peak(freq=data_freq, data=data_ZTR) # Z12 공진주파수


In [ ]:
import numpy as np
isinstance(freq1, np.ndarray)

float(freq1[0])







In [ ]:
design1.export_convergence(setup="Setup1")

In [ ]:
pass_number, tetrahedra, delta_S = design1.get_report_data(setup="Setup1")

In [ ]:
design1.export_convergence(setup="Setup1", variations="", output_file=None)
    

In [ ]:
import pandas as pd
import time

design = design1
project_dir = project1.path

result_expressions = []
result_expressions.append(f"mag(Zt(Tx_port_T1,Tx_port_T1))")
result_expressions.append(f"mag(Zt(Rx_port_T1,Rx_port_T1))")
result_expressions.append(f"mag(Zt(Tx_port_T1,Rx_port_T1))")
result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))/sqrt(im(Zt(Tx_port_T1,Tx_port_T1))*im(Zt(Rx_port_T1,Rx_port_T1)))")

report1 = design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                report_category="Terminal Solution Data", plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="Resonance Data")
time.sleep(1)
sim_data1 = design.post.export_report_to_csv(project_dir=project_dir, plot_name=report1.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

In [ ]:
result_expression

In [ ]:
import pandas as pd

design = design1
project_dir = project1.path

result_expressions = []
# result_expressions.append(f"im(Zt(Tx_port_T1,Tx_port_T1))/2/pi/freq")
# result_expressions.append(f"im(Zt(Rx_port_T1,Rx_port_T1))/2/pi/freq")
# result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))/2/pi/freq")
result_expressions.append(f"im(Zt(Tx_port_T1,Tx_port_T1))/(2*pi*freq)")
result_expressions.append(f"im(Zt(Rx_port_T1,Rx_port_T1))/(2*pi*freq)")
result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))/(2*pi*freq)")

report = design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                report_category="Terminal Solution Data", plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name=None)
sim_data = design.post.export_report_to_csv(project_dir=project_dir, plot_name=report.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)


In [ ]:
data = pd.read_csv(sim_data)
data = design.post_processing.data_preprocessing(data)

data_freq = data["Freq [Hz]"]
data_LTT = data["im(Zt(Tx_port_T1,Tx_port_T1))/2/pi/freq []"]
data_LRR = data["im(Zt(Rx_port_T1,Rx_port_T1))/2/pi/freq []"]
data_LTR = data["im(Zt(Tx_port_T1,Rx_port_T1))/2/pi/freq []"]

In [ ]:
report = design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                report_category="Terminal Solution Data", plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="inductance data5")

In [ ]:
report

In [ ]:
result_expressions = []
result_expressions.append(f"mag(Zt(Tx_port_T1,Tx_port_T1))")
result_expressions.append(f"mag(Zt(Rx_port_T1,Rx_port_T1))")
result_expressions.append(f"mag(Zt(Tx_port_T1,Rx_port_T1))")

report1 = design1.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="impedance_mag data")

project_dir = project1.path

sim_data1 = design1.post.export_report_to_csv(project_dir=project_dir, plot_name=report1.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

In [ ]:
import pandas as pd

design = design1
project_dir = project1.path

result_expressions = []
result_expressions.append(f"dB(St(Tx_port_T1,Tx_port_T1))")
result_expressions.append(f"dB(St(Rx_port_T1,Rx_port_T1))")
result_expressions.append(f"dB(St(Tx_port_T1,Rx_port_T1))")

report = design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="return loss data")
sim_data = design.post.export_report_to_csv(project_dir=project_dir, plot_name=report.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
data = pd.read_csv(sim_data)
data = design.post_processing.data_preprocessing(data)

data_freq = data["Freq [Hz]"]
data_S11 = data["dB(St(Tx_port_T1,Tx_port_T1)) []"]
data_S22 = data["dB(St(Rx_port_T1,Rx_port_T1)) []"]
data_S12 = data["dB(St(Tx_port_T1,Rx_port_T1)) []"]



freq1, peak1 = design.post_processing.detect_max(freq=data_freq, data=abs(data_S11))
freq2, peak2 = design.post_processing.detect_max(freq=data_freq, data=abs(data_S22))
freq3, peak3 = design.post_processing.detect_min(freq=data_freq, data=abs(data_S12))

print(f"S11_min_freq : {freq1} [Hz], S11_min_Z : {peak1} [dB]")
print(f"S22_max_freq : {freq2} [Hz], S22_max_Z : {peak2} [dB]")
print(f"S12_max_freq : {freq3} [Hz], S12_max_Z : {peak3} [dB]")


In [ ]:
report

In [ ]:
data1 = design1.post_processing.data_preprocessing(data1)

data1

In [ ]:
data1_freq = data1["Freq [Hz]"]
data1_ZTT = data1["mag(Zt(Tx_port_T1,Tx_port_T1)) []"]
data1_ZRR = data1["mag(Zt(Rx_port_T1,Rx_port_T1)) []"]
data1_ZTR = data1["mag(Zt(Tx_port_T1,Rx_port_T1)) []"]


freq1, peak1 = design1.post_processing.detect_peak(freq=data1_freq, data=data1_ZTT)
freq2, peak2 = design1.post_processing.detect_peak(freq=data1_freq, data=data1_ZRR)
freq3, peak3 = design1.post_processing.detect_peak(freq=data1_freq, data=data1_ZTR)

print(f"TT_resonant_freq : {freq1} [Hz], TT_resonant_Z : {peak1} [Ω]")
print(f"RR_resonant_freq : {freq2} [Hz], RR_resonant_Z : {peak2} [Ω]")
print(f"TR_resonant_freq : {freq3} [Hz], TR_resonant_Z : {peak3} [Ω]")

In [ ]:
design1.GetModule("AnalysisSetup")

In [ ]:
setup = design1.create_setup("Setup1")

In [ ]:
design = design1

region = design.modeler.create_air_region(x_pos="175",x_neg="175", y_pos="175",y_neg="175", z_pos = "2500", z_neg="2500")
design.assign_material(assignment=region, material="vacuum")
design.assign_radiation_boundary_to_objects(assignment=region, name="Radiation")


In [ ]:
def create_PCB(design) :


    if design.variables["Tx_outer_x"] > design.variables["Rx_outer_x"] :
        outer_x = "(Tx_outer_x)"
    else :
        outer_x = "(Rx_outer_x)"
    
    if design.variables["Tx_outer_y"] > design.variables["Rx_outer_y"] :
        outer_y = "(Tx_outer_y)"
    else :
        outer_y = "(Rx_outer_y)"

    origin_z = f"Tx_Rx_gap/2 + Rx_layer*PCB_thickness + (Rx_layer-1)*Rx_Rx_gap"
    dimension_z = f"({origin_z}) + Tx_Rx_gap/2 + Tx_layer*PCB_thickness + (Tx_layer-1)*Tx_Tx_gap"

    origin = [f"-(1.4*{outer_x})", f"-(1.4*{outer_y})", f"-({origin_z})"]
    dimension = [f"2*(1.4*{outer_x})", f"2*(1.4*{outer_y})", f"({dimension_z})"]

    print(f"({origin_z})")

    PCB = design.modeler.create_box(
        origin = origin,
        sizes = dimension,
        name = "PCB",
        material = "FR4_epoxy"
    )

    return PCB


create_PCB(design1)




In [ ]:
abs(data_S11)

In [ ]:
freq1, peak1 = design.post_processing.detect_peak(freq=data_freq, data=abs(data_S12), prominence=30) # Z11 공진 주파수

peak1

In [ ]:




if self.Tx_outer_x > self.Rx_outer_x :
    outer_x = "Tx_outer_x"
else :
    outer_x = "Rx_outer_x"

if self.Tx_outer_y > self.Rx_outer_y :
    outer_y = "Tx_outer_y"
else :
    outer_y = "Rx_outer_y"   

origin = [f"-(1.4*{outer_x})", f"-(1.4*{outer_y})", f"-{(self.Tx_preg/2+self.Tx_thick+self.Tx_Rx_gap+2*self.Rx_thick+self.Rx_preg)}mm"]
dimension = [f"2*(1.4*{outer_x})", f"2*(1.4*{outer_y})", f"{(self.Tx_preg+2*self.Tx_thick+self.Tx_Rx_gap+2*self.Rx_thick+self.Rx_preg)}mm"]
PCB = design.modeler.create_box(
    origin = origin,
    sizes = dimension,
    name = "PCB",
    material = "FR4_epoxy"
)

PCB.color = [0 ,128 ,64]
PCB.transparency = 0

design.modeler.subtract(blank_list=PCB, tool_list=[coil_Tx, coil_Rx, Tx_ter1, Tx_ter2, Rx_ter1, Rx_ter2], keep_originals=True)

In [ ]:


sim1.input

In [ ]:
Tx_port.child_object.

In [ ]:
a = [1,2,3,4]
a.insert(2,55)

a

In [ ]:
turns = 8


if turns % 2 == 1 :
    turns = math.ceil(int(turns)/2)
    turns_sub = 1
else :
    turns = math.ceil(int(turns)/2)
    turns_sub = 0

print(f"turns : {turns} {turns_sub}")

In [ ]:
import math
math.ceil(3/2)

In [ ]:
int(13/2)

In [ ]:




def create_model(design, name, winding=1, up=True,*args, **kwargs) :

    color = kwargs.get("color", None)

    turns = kwargs.get("turns", None)

    outer_x = kwargs.get("outer_x", None)
    outer_y = kwargs.get("outer_y", None)
    fillet = kwargs.get("fillet", None)
    inner = kwargs.get("inner", None)
    fill_factor = kwargs.get("fill_factor", None)
    theta1 = kwargs.get("theta1", None)
    theta2 = kwargs.get("theta2", None)

    height = kwargs.get("height", None)
    move = kwargs.get("move", None)

    Tx_Rx_gap = kwargs.get("Tx_Rx_gap", None)


    

    if winding == 1 :

        winding_variables = {"outer_x": outer_x, "outer_y": outer_y, "fillet": fillet, "inner": inner, "fill_factor": fill_factor, "theta1": theta1, "theta2": theta2}
        coil = design.model3d.winding.coil_points(turns=turns, **winding_variables)
        coil.points.insert(0, [f"0mm", f"{coil.points[0][1]}", coil.points[0][2]])


        winding = design.model3d.winding.create_polyline(name=name, points=coil.points, width=coil.width, height=height)
        winding.move([0,0,f"-{move}/2"])

        if up :
            point1 = [f"{coil.points[0][0]}-{coil.width}/2", coil.points[0][1], f"({coil.points[0][2]} - ({height})/2)"]
            point2 = [f"{coil.points[0][0]}-{coil.width}/2", coil.points[0][1], f"({coil.points[0][2]} + ({sim1.Tx_preg}) + 3*({height})/2)mm"]
            ter1 = design.model3d.winding.create_polyline(name=f"{name}_ter1", points=[point1, point2], width=coil.width, height=coil.width)
            point1 = [f"{coil.points[-1][0]}+{coil.width}/2", coil.points[-1][1], f"({coil.points[-1][2]} - ({height})/2)"]
            point2 = [f"{coil.points[-1][0]}+{coil.width}/2", coil.points[-1][1], f"({coil.points[-1][2]} + ({sim1.Tx_preg}) + 3*({height})/2)mm"]
            ter2 = design.model3d.winding.create_polyline(name=f"{name}_ter2", points=[point1, point2], width=coil.width, height=coil.width)

            ter_edge1 = ter1.top_face_y.top_edge_z
            ter_edge2 = ter2.bottom_face_y.top_edge_z

        else :
            point1 = [f"{coil.points[0][0]}-{coil.width}/2", coil.points[0][1], f"({coil.points[0][2]} + ({height})/2)"]
            point2 = [f"{coil.points[0][0]}-{coil.width}/2", coil.points[0][1], f"({coil.points[0][2]} - ({sim1.Tx_preg}) + 3*({height})/2)mm"]
            ter1 = design.model3d.winding.create_polyline(name=f"{name}_ter1", points=[point1, point2], width=coil.width, height=coil.width)
            point1 = [f"{coil.points[-1][0]}+{coil.width}/2", coil.points[-1][1], f"({coil.points[-1][2]} + ({height})/2)"]
            point2 = [f"{coil.points[-1][0]}+{coil.width}/2", coil.points[-1][1], f"({coil.points[-1][2]} - ({sim1.Tx_preg}) + 3*({height})/2)mm"]
            ter2 = design.model3d.winding.create_polyline(name=f"{name}_ter2", points=[point1, point2], width=coil.width, height=coil.width)

            ter_edge1 = ter1.bottom_face_y.bottom_edge_z
            ter_edge2 = ter2.top_face_y.bottom_edge_z


        ter1.material_name = "pec"
        ter2.material_name = "pec"

        ter1.move([0,0,f"-{move}/2"])
        ter2.move([0,0,f"-{move}/2"])


        ter_edges = design.modeler.create_object_from_edge([ter_edge1, ter_edge2], non_model=False)
        ter_face = design.modeler.connect(ter_edges)

    else :
        
        winding_variables = {"outer_x": outer_x, "outer_y": outer_y, "fillet": fillet, "inner": inner, "fill_factor": fill_factor, "theta1": theta1, "theta2": theta2, "turns_sub": 1}
        coil = design.model3d.winding.coil_points(turns=turns, **winding_variables)
        coil.points.insert(0, [f"{coil.points[0][0]}", f"{coil.points[0][1]} - 1mm", coil.points[0][2]])

        winding1 = design.model3d.winding.create_polyline(name=f"{name}_1", points=coil.points, width=coil.width, height=height)
        winding1.move([0,0,f"(thick+preg)/2"])


        winding_variables = {"outer_x": outer_x, "outer_y": outer_y, "fillet": fillet, "inner": inner, "fill_factor": fill_factor, "theta1": theta1, "theta2": theta2, "turns_sub": 0}
        coil = design.model3d.winding.coil_points(turns=turns, **winding_variables)
        coil.points.insert(0, [f"{coil.points[0][0]}", f"{coil.points[0][1]} - 1mm", coil.points[0][2]])

        winding1 = design.model3d.winding.create_polyline(name=f"{name}_1", points=coil.points, width=coil.width, height=height)
        winding1.move([0,0,f"(thick+preg)/2"])


        # via1 = design.model3d.winding.create_via(name=f"{name}_via", center=coil.points[-1], outer_R=f"0.8*{coil.width}/2", inner_R=f"0.6*{coil.width}/2", height=f"{2*sim1.Rx_thick+sim1.Rx_preg}mm", via_pad_R=f"1.2*{coil.width}/2", via_pad_thick=f"{sim1.Rx_thick}mm")
        # via2 = design.model3d.winding.create_via(name=f"{name}_sub_via", center=coil.points[-1], outer_R=f"0.6*{coil.width}/2", height="1mm*2")
        # via1.move([0,0,-(sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])
        # via2.move([0,0,-(sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])
        
        # (sim1.Rx_thick+sim1.Rx_preg)/2 - (sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])


    winding.color = color
    winding.transparency = 0
    winding.material_name = "copper"

    port = design.lumped_port(assignment=ter_face[0], reference=[ter1], create_port_sheet=False, port_on_plane=True, integration_line=0, impedance=50, name=f"{name}_port", renormalize=True, deembed=False, terminals_rename=True)


coil_variable = {
    "color": [255, 10, 10],
    "turns": design1.variables["Tx_turns"],
    "outer_x": "Tx_outer_x",
    "outer_y": "Tx_outer_y",
    "fillet": "Tx_fillet",
    "inner": "Tx_inner",
    "fill_factor": "Tx_fill_factor",
    "theta1": "Tx_theta1",
    "theta2": "Tx_theta2",
    "height": "thick",
    # "move": "(-(thick+preg)/2)",
    "move": "0",
    "Tx_Rx_gap": "Tx_Rx_gap",
    "preg": "preg"
}


create_model(design1, name="Tx", winding=2, up=True, **coil_variable)

In [ ]:
design1.variables["Tx_turns"]

In [ ]:
import copy

def create_model(design) :

    # Tx

    winding_variables = {"outer_x": "Tx_outer_x", "outer_y": "Tx_outer_y", "fillet": "Tx_fillet", "inner": "Tx_inner", "fill_factor": "Tx_fill_factor", "theta1": "Tx_theta1", "theta2": "Tx_theta2"}
    Tx_coil1 = design.model3d.winding.coil_points(turns=sim1.Tx_turns, **winding_variables)
    Tx_coil1.points.insert(0, [f"0mm", f"{Tx_coil1.points[0][1]}", Tx_coil1.points[0][2]])

    coil_Tx1 = design.model3d.winding.create_polyline(name="Tx_winding1", points=Tx_coil1.points, width=Tx_coil1.width, height=f"{sim1.Tx_thick}mm")
    sim1.coil = coil_Tx1

    # coil_Tx1_box = design.model3d.create_box(origin=Tx_coil1[0], )

    coil_Tx1.move([0,0,-(sim1.Tx_thick+sim1.Tx_preg)/2])

    # Tx_coil2 = copy.deepcopy(Tx_coil1)

    # Tx_coil2.points[-1] = [f"({Tx_coil2.points[-1][0]}) - {sim1.Tx_shift}*({Tx_coil2.wg})",f"({Tx_coil2.points[-1][1]})",f"{Tx_coil2.points[-1][2]}"]
    # Tx_coil2.points.append([f"({Tx_coil2.points[-1][0]})",f"({Tx_coil2.points[-1][1]}) + {sim1.Tx_shift}*({Tx_coil2.wg})",f"{Tx_coil2.points[-1][2]}"])
    # Tx_coil2.points[0] = [f"({Tx_coil2.points[0][0]})",f"({Tx_coil2.points[0][1]}) + {sim1.Tx_shift}*({Tx_coil2.wg})",f"{Tx_coil2.points[0][2]}"]

    # coil_Tx2 = design.model3d.winding.create_polyline(name="Tx_winding2", points=Tx_coil2.points, width=Tx_coil2.width, height=f"{sim1.Tx_thick}mm")
    # coil_Tx2.mirror(origin=[0,0,0], vector=[1,0,0])
    # coil_Tx2.move([f"-{sim1.Tx_shift}*({Tx_coil2.wg})",f"-{sim1.Tx_shift}*({Tx_coil2.wg})",-(sim1.Tx_thick+sim1.Tx_preg)/2])

    # coil_Tx = design.model3d.unite(assignment=[coil_Tx1, coil_Tx2, Tx_via1])
    # design.modeler.subtract(blank_list=coil_Tx, tool_list=Tx_via2, keep_originals=False)

    coil_Tx = coil_Tx1

    coil_Tx.transparency = 0
    coil_Tx.color = [255, 10, 10]
    coil_Tx.material_name = "copper"


    point1 = [f"{Tx_coil1.points[0][0]}-{Tx_coil1.width}/2", Tx_coil1.points[0][1], f"({Tx_coil1.points[0][2]} - ({sim1.Tx_thick})/2)mm"]
    point2 = [f"{Tx_coil1.points[0][0]}-{Tx_coil1.width}/2", Tx_coil1.points[0][1], f"({Tx_coil1.points[0][2]} + ({sim1.Tx_preg}) + 3*({sim1.Tx_thick})/2)mm"]
    Tx_ter1 = design.model3d.winding.create_polyline(name="Tx_ter1", points=[point1, point2], width=Tx_coil1.width, height=Tx_coil1.width)
    point1 = [f"{Tx_coil1.points[-1][0]}+{Tx_coil1.width}/2", Tx_coil1.points[-1][1], f"({Tx_coil1.points[-1][2]} - ({sim1.Tx_thick})/2)mm"]
    point2 = [f"{Tx_coil1.points[-1][0]}+{Tx_coil1.width}/2", Tx_coil1.points[-1][1], f"({Tx_coil1.points[-1][2]} + ({sim1.Tx_preg}) + 3*({sim1.Tx_thick})/2)mm"]
    Tx_ter2 = design.model3d.winding.create_polyline(name="Tx_ter2", points=[point1, point2], width=Tx_coil1.width, height=Tx_coil1.width)

    Tx_ter1.material_name = "pec"
    Tx_ter2.material_name = "pec"

    
    Tx_ter1.move([0,0,-(sim1.Tx_thick+sim1.Tx_preg)/2])
    Tx_ter2.move([0,0,-(sim1.Tx_thick+sim1.Tx_preg)/2])


    Tx_ter_edge1 = Tx_ter1.top_face_y.top_edge_z
    Tx_ter_edge2 = Tx_ter2.bottom_face_y.top_edge_z

    Tx_ter_edges = design.modeler.create_object_from_edge([Tx_ter_edge1, Tx_ter_edge2], non_model=False)
    Tx_ter_face = design.modeler.connect(Tx_ter_edges)

    Tx_port = design.lumped_port(assignment=Tx_ter_face[0], reference=[Tx_ter1], create_port_sheet=False, port_on_plane=True, integration_line=0, impedance=50, name="Tx_port", renormalize=True, deembed=False, terminals_rename=True)



    # Rx

    winding_variables = {"outer_x": "Rx_outer_x", "outer_y": "Rx_outer_y", "fillet": "Rx_fillet", "inner": "Rx_inner", "fill_factor": "Rx_fill_factor", "theta1": "Rx_theta1", "theta2": "Rx_theta2"}
    Rx_coil1 = design.model3d.winding.coil_points(turns=sim1.Rx_turns, **winding_variables)
    Rx_coil1.points.insert(0, [f"{Rx_coil1.points[0][0]}", f"{Rx_coil1.points[0][1]} - 1mm", Rx_coil1.points[0][2]])

    Rx_via1 = design.model3d.winding.create_via(name="Rx_via", center=Rx_coil1.points[-1], outer_R=f"0.8*{Rx_coil1.width}/2", inner_R=f"0.6*{Rx_coil1.width}/2", height=f"{2*sim1.Rx_thick+sim1.Rx_preg}mm", via_pad_R=f"1.2*{Rx_coil1.width}/2", via_pad_thick=f"{sim1.Rx_thick}mm")
    Rx_via2 = design.model3d.winding.create_via(name="Rx_sub_via", center=Rx_coil1.points[-1], outer_R=f"0.6*{Rx_coil1.width}/2", height="1mm*2")
    Rx_via1.move([0,0,-(sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])
    Rx_via2.move([0,0,-(sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])

    coil_Rx1 = design.model3d.winding.create_polyline(name="Rx_winding1", points=Rx_coil1.points, width=Rx_coil1.width, height=f"{sim1.Rx_thick}mm")
    coil_Rx1.move([0,0,(sim1.Rx_thick+sim1.Rx_preg)/2 - (sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])

    Rx_coil2 = copy.deepcopy(Rx_coil1)

    Rx_coil2.points[-1] = [f"({Rx_coil2.points[-1][0]}) - {sim1.Rx_shift}*({Rx_coil2.wg})",f"({Rx_coil2.points[-1][1]})",f"{Rx_coil2.points[-1][2]}"]
    Rx_coil2.points.append([f"({Rx_coil2.points[-1][0]})",f"({Rx_coil2.points[-1][1]}) + {sim1.Rx_shift}*({Rx_coil2.wg})",f"{Rx_coil2.points[-1][2]}"])
    Rx_coil2.points[0] = [f"({Rx_coil2.points[0][0]})",f"({Rx_coil2.points[0][1]}) + {sim1.Rx_shift}*({Rx_coil2.wg})",f"{Rx_coil2.points[0][2]}"]

    coil_Rx2 = design.model3d.winding.create_polyline(name="Rx_winding2", points=Rx_coil2.points, width=Rx_coil2.width, height=f"{sim1.Rx_thick}mm")
    coil_Rx2.mirror(origin=[0,0,0], vector=[1,0,0])
    coil_Rx2.move([f"-{sim1.Rx_shift}*({Rx_coil2.wg})",f"-{sim1.Rx_shift}*({Rx_coil2.wg})",-(sim1.Rx_thick+sim1.Rx_preg)/2 - (sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])

    coil_Rx = design.model3d.unite(assignment=[coil_Rx1, coil_Rx2, Rx_via1])
    design.modeler.subtract(blank_list=coil_Rx, tool_list=Rx_via2, keep_originals=False)

    coil_Rx.transparency = 0
    coil_Rx.color = [10, 10, 255]
    coil_Rx.material_name = "copper"


    point1 = Rx_coil1.points[0]
    point2 = [point1[0], f"({point1[1]})-0.05mm", point1[2]]
    Rx_ter1 = design.model3d.winding.create_polyline(name="Rx_ter1", points=[point1, point2], width=Rx_coil1.width, height=f"{sim1.Rx_thick}mm")
    Rx_ter2 = design.model3d.winding.create_polyline(name="Rx_ter2", points=[point1, point2], width=Rx_coil1.width, height=f"{sim1.Rx_thick}mm")

    Rx_ter1.material_name = "pec"
    Rx_ter2.material_name = "pec"

    Rx_ter1.mirror(origin=[0,0,0], vector=[1,0,0])
    Rx_ter1.move([f"-{sim1.Rx_shift}*({Rx_coil2.wg})",0,-(sim1.Rx_thick+sim1.Rx_preg)/2 - (sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])
    Rx_ter2.move([0,0,(sim1.Rx_thick+sim1.Rx_preg)/2 - (sim1.Tx_Rx_gap+sim1.Tx_thick+sim1.Rx_thick+sim1.Tx_preg/2+sim1.Rx_preg/2)])


    Rx_ter_edge1 = Rx_ter1.bottom_face_y.top_edge_x
    Rx_ter_edge2 = Rx_ter2.bottom_face_y.bottom_edge_x

    Rx_ter_edges = design.modeler.create_object_from_edge([Rx_ter_edge1, Rx_ter_edge2], non_model=False)
    Rx_ter_face = design.modeler.connect(Rx_ter_edges)

    Rx_port = design.lumped_port(assignment=Rx_ter_face[0], reference=[Rx_ter1], create_port_sheet=False, port_on_plane=True, integration_line=0, impedance=50, name="Rx_port", renormalize=True, deembed=False, terminals_rename=True)




    # PCB

    if sim1.Tx_outer_x > sim1.Rx_outer_x :
        outer_x = "Tx_outer_x"
    else :
        outer_x = "Rx_outer_x"

    if sim1.Tx_outer_y > sim1.Rx_outer_y :
        outer_y = "Tx_outer_y"
    else :
        outer_y = "Rx_outer_y"   

    origin = [f"-(1.4*{outer_x})", f"-(1.4*{outer_y})", f"-{(sim1.Tx_preg/2+sim1.Tx_thick+sim1.Tx_Rx_gap+2*sim1.Rx_thick+sim1.Rx_preg)}mm"]
    dimension = [f"2*(1.4*{outer_x})", f"2*(1.4*{outer_y})", f"{(sim1.Tx_preg+2*sim1.Tx_thick+sim1.Tx_Rx_gap+2*sim1.Rx_thick+sim1.Rx_preg)}mm"]
    PCB = design.modeler.create_box(
        origin = origin,
        sizes = dimension,
        name = "PCB",
        material = "FR4_epoxy"
    )

    PCB.color = [0 ,128 ,64]
    PCB.transparency = 0

    design.modeler.subtract(blank_list=PCB, tool_list=[coil_Tx, coil_Rx, Tx_ter1, Tx_ter2, Rx_ter1, Rx_ter2], keep_originals=True)



    region = design.modeler.create_air_region(x_pos="175",x_neg="175", y_pos="175",y_neg="175", z_pos = "3500", z_neg="3500")
    design.assign_material(assignment=region, material="vacuum")
    design.assign_radiation_boundary_to_objects(assignment=region, name="Radiation")


create_model(design1)

In [ ]:
sim1.desktop.projects

In [ ]:
sim1.desktop.delete()

In [ ]:
sim1.desktop.close_desktop()

In [ ]:
project1.delete()

In [ ]:
project2.delete()

In [ ]:
project_names = [project.name for project in sim1.desktop.projects]

project_names

In [ ]:
dir(sim1.desktop)

In [ ]:
sim1.desktop.kill_process()

In [ ]:
sim1.desktop.projects[0].name




In [ ]:
project_names = [project.name for project in sim1.desktop.projects]

project_names

In [ ]:
dir(design1.solver_instance)

In [ ]:
design1.solver_instance.odesign.GetName()

In [ ]:
design1.solver_instance.odesign.GetVariables()

In [ ]:
import inspect

# 소스 코드 확인 (가능한 경우)
try:
    print(inspect.getsource(design.SetVariableValue))
except OSError:
    print("소스 코드를 가져올 수 없습니다 (C extension 또는 동적 생성)")

In [ ]:
design = project.SetActiveDesign("sim4")

design.

In [ ]:
varaible_manager = design._variable_manager

design["w"] = "10mm"

In [ ]:
design4["a"] = "1mm"
design4["b"] = "2mm"
design4["c"] = "3mm"
design4["d"] = "4mm"
design4["e"] = "5mm"
design4["f"] = "6mm"
design4["g"] = "7mm"
design4["h"] = "8mm"

In [ ]:
for design in project.designs:

    print(design.name)
    print(design.solver)
    print(design.solution)
    print(design.variables)
    print("")

In [ ]:
design = project.designs["sim4"]

In [ ]:
dir(design.variable_manager)

In [ ]:
v = design.GetVariables()

v

In [ ]:
design.GetVariableValue("a")

In [ ]:
design.variables

In [ ]:
# 한 줄로 만들기
variables = {
    var_name: design.GetVariableValue(var_name) 
    for var_name in design.GetVariables()
}

print(variables)

In [ ]:
dir(design)

In [ ]:
sim1.desktop.project_list

project = sim1.desktop.projects[project.name]

dir(project.designs[0])

In [ ]:
project = sim1.desktop.projects[0]

design = project.designs[0]

design.variables

In [ ]:
sim1.create_design(name="HFSS_design1")
sim1.create_design(name="HFSS_design2")
sim1.create_design(name="HFSS_design3")


In [ ]:
project = sim1.desktop.create_project(name="test10")

In [ ]:
import os
project_path = pyaedt.generate_unique_project_name()
project_name = os.path.splitext(os.path.basename(project_path))[0]
project_name

In [ ]:
import os
import pyaedt

path = os.path.join(os.getcwd(), "simulation_test7")
name = "simulation_test3"
pathh = path
namee = name

print(path)
print(name)

# name 기본값 설정
if name is None:
    name = pyaedt.generate_unique_project_name() # 해당 값의 포맷 예시 : 'C:\\Users\\peets\\AppData\\Local\\Temp\\pyaedt_prj_COM\\Project_QWI.aedt'
    name = os.path.splitext(os.path.basename(name))[0]
# name이 .aedt로 끝나면 확장자를 떼어낸다
if name.endswith('.aedt'):
    name = os.path.splitext(name)[0]
# path 처리: .aedt 확장자가 있으면 그대로 사용, 없으면 path/name.aedt 형태로 만들기
if path is None:
    path = os.getcwd()


if path.endswith('.aedt'):
    # if path format : ~~/~~/~~/simulation.aedt
    name = os.path.splitext(os.path.basename(path))[0]
elif path.endswith(name):
    # if path format : ~~/~~/~~/simulation
    path = path + ".aedt"
elif not path.endswith('.aedt'):
    # if path format : ~~/~~/~~
    path = os.path.join(path, name + ".aedt")
else:
    pass

print(path)
print(name)


path = pathh
name = namee


project = sim1.desktop.create_project(path=path, name=name)

In [ ]:
project.aedt_path

In [ ]:
project = sim1.desktop.create_project(path=os.path.join(os.getcwd(), "test_simulation"),name="test11")

In [ ]:
project.aedt_path

In [ ]:
sim1.desktop.project_list

In [ ]:
sim1.desktop.od

In [ ]:
import os
path = os.path.join(os.getcwd(), "test_simulation.aedt")

name = os.path.splitext(os.path.basename(path))[0]

name

In [ ]:
projects = {}
project_names = sim1.desktop.project_list

project_names


In [ ]:


for project_name in project_names:
project = pyProject(self, name=project_name, load=True)
projects[project_name] = project


In [ ]:
sim1.desktop.get_project_list()[0].GetName()

In [ ]:
import os



projects


    

In [ ]:
project = sim1.desktop.odesktop.SetActiveProject('insulation_amp_2026')

In [ ]:
project.GetPath()

project.GetName()

In [ ]:
sim1.desktop.SetActiveProject('Project11')

In [ ]:
sim1.oproject

In [ ]:

importlib.reload(simulation_script)
sim1.__class__ = simulation_script.Simulation
sim1.debug2()



In [ ]:
sim1.project = sim1.desktop.create_project(name=sim1.PROJECT_NAME)

In [ ]:
sim1.design = sim1.project.create_design(name="HFSS_design", solver="HFSS", solution=None)

In [ ]:
import sys
import traceback
import logging
import portalocker
# import fcntl

sys.path.insert(0, r"/gpfs/home1/r1jae262/jupyter/git/pyaedt_library/src/")
sys.path.insert(0, r"D:/git/pyaedt_library/src/")

import pyaedt_module
from pyaedt_module.core import pyDesktop
import os
import time
from datetime import datetime

import math
import copy

import pandas as pd

import platform
import csv

from scipy.signal import find_peaks




class Simulation() :

    def __init__(self) :

        self.NUM_CORE = 1
        self.NUM_TASK = 1

        
        os_name = platform.system()
        if os_name == "Windows":
            GUI = False
        else :
            GUI = True

        self.desktop = pyDesktop(version=None, non_graphical=GUI)

        file_path = "simulation_num.txt"

        # 파일이 존재하지 않으면 생성
        if not os.path.exists(file_path):
            with open(file_path, "w", encoding="utf-8") as file:
                file.write("1")

        # 읽기/쓰기 모드로 파일 열기
        with open(file_path, "r+", encoding="utf-8") as file:
            # 파일 잠금: LOCK_EX는 배타적 잠금,  블로킹 모드로 실행 (Windows/Linux 호환)
            portalocker.lock(file, portalocker.LOCK_EX)

            # 파일에서 값 읽기
            content = int(file.read().strip())
            self.num = content
            self.PROJECT_NAME = f"simulation{content}"
            content += 1

            # 파일 포인터를 처음으로 되돌리고, 파일 내용 초기화 후 새 값 쓰기
            file.seek(0)
            file.truncate()
            file.write(str(content))

            # 파일은 with 블록 종료 시 자동으로 닫히며, 잠금도 해제됨



        log_simulation(number=self.num, pid=self.desktop.pid)




        # simulation_log 폴더에 로그 파일 생성 및 FileHandler 설정
        log_folder = "simulation_log"
        os.makedirs(log_folder, exist_ok=True)
        log_file_path = os.path.join(log_folder, f"{self.PROJECT_NAME}_log.txt")

        # FileHandler를 생성해서 로거에 추가
        file_handler = logging.FileHandler(log_file_path, mode="a", encoding="utf-8")
        file_handler.setLevel(logging.DEBUG)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        # root logger 또는 원하는 로거에 핸들러 추가
        logger = logging.getLogger()  # 또는 logging.getLogger(self.PROJECT_NAME)
        logger.setLevel(logging.DEBUG)
        logger.addHandler(file_handler)


    def simulation(self):
        self.start_time = time.time()
        

        

        self.create_design(name="HFSS_design")
        print("manual log : set variable start")
        self.set_variable(self.design)
        print("manual log : create model start")
        self.create_model(self.design)
        print("manual log : analyze start")
        self.analyze()


    def create_design(self, name) : 
    
        self.project = self.desktop.create_project()
        self.design = self.project.create_design(name=name, solver="HFSS", solution=None)


    
    def detect_Speak(self, freq=None, data=None) :

        peaks_prom, properties = find_peaks(data, prominence=1)

        if freq is not None :
            peak_freqs_prom = freq[peaks_prom].values
            peak_vals_prom = data[peaks_prom].values
            return peak_freqs_prom, peak_vals_prom
        
        elif freq is None :
            peak_vals_prom = data[peaks_prom].values
            return peak_vals_prom
        

    def set_variable(self, design) :

        

        # self.Tx_outer_x = design.random_variable(variable_name="Tx_outer_x", lower=1.5, upper=3, resolution=0.1, unit="mm")
        self.Tx_outer_x = design.random_variable(variable_name="Tx_outer_x", lower=0.8, upper=2.5, resolution=0.1, unit="mm")
        self.Tx_ratio = design.random_variable(lower=0.4, upper=0.9, resolution=0.01)
        # design["Tx_outer_y"] = f"{self.Tx_ratio} * {self.Tx_outer_x}mm"
        self.Tx_outer_y = self.Tx_ratio * self.Tx_outer_x

        Tx_inner_min = 0.4*min(self.Tx_outer_x, self.Tx_outer_y)
        Tx_inner_max = 0.8*min(self.Tx_outer_x, self.Tx_outer_y)
        self.Tx_inner = design.random_variable(variable_name="Tx_inner", lower=Tx_inner_min, upper=Tx_inner_max, resolution=0.01, unit="mm")

        Tx_fillet_min = self.Tx_inner * (math.tan(75 * math.pi/180)  - 1) / math.tan(75 * math.pi/180)  * 1.1
        Tx_fillet_max = self.Tx_inner * 0.8 if self.Tx_inner * 0.8 > Tx_fillet_min else self.Tx_inner * 0.9
        self.Tx_fillet = design.random_variable(variable_name="Tx_fillet", lower=Tx_fillet_min, upper=Tx_fillet_max, resolution=0.01, unit="mm")

        self.Tx_fill_factor = design.random_variable(variable_name="Tx_fill_factor", lower=0.3, upper=0.8, resolution=0.01, unit=None)

        # design["Tx_theta1"] = "15*pi/180"
        # design["Tx_theta2"] = "75*pi/180"


        self.Tx_Rx_ratio = design.random_variable(lower=0.6, upper=1.5, resolution=0.01)
        # design["Rx_outer_x"] = f"{self.Tx_outer_x} * {self.Tx_Rx_ratio}mm"
        self.Rx_outer_x = self.Tx_outer_x * self.Tx_Rx_ratio

        self.Rx_ratio = design.random_variable(lower=0.4, upper=0.9, resolution=0.01)
        # design["Rx_outer_y"] = f"{self.Rx_ratio} * {self.Rx_outer_x}mm"
        self.Rx_outer_y = self.Rx_ratio * self.Rx_outer_x

        Rx_inner_min = 0.4*min(self.Rx_outer_x, self.Rx_outer_y)
        Rx_inner_max = 0.8*min(self.Rx_outer_x, self.Rx_outer_y)
        self.Rx_inner = design.random_variable(variable_name="Rx_inner", lower=Rx_inner_min, upper=Rx_inner_max, resolution=0.01, unit="mm")

        Rx_fillet_min = self.Rx_inner * (math.tan(75 * math.pi/180)  - 1) / math.tan(75 * math.pi/180)  * 1.1
        Rx_fillet_max = self.Rx_inner * 0.8 if self.Rx_inner * 0.8 > Rx_fillet_min else self.Rx_inner * 0.9
        self.Rx_fillet = design.random_variable(variable_name="Rx_fillet", lower=Rx_fillet_min, upper=Rx_fillet_max, resolution=0.01, unit="mm")

        self.Rx_fill_factor = design.random_variable(variable_name="Rx_fill_factor", lower=0.3, upper=0.8, resolution=0.01, unit=None)

        # design["Rx_theta1"] = "15*pi/180"
        # design["Rx_theta2"] = "75*pi/180"


        # self.Tx_turns = design.random_variable(variable_name=None, lower=4, upper=8, resolution=1)
        self.Tx_turns = design.random_variable(variable_name=None, lower=5, upper=12, resolution=1)
        # self.Rx_turns = design.random_variable(variable_name=None, lower=4, upper=15, resolution=1)
        # self.Rx_turns = int(round(self.Tx_turns * design.random_variable(variable_name=None, lower=1.3, upper=1.7, resolution=0.01), 0))
        self.Rx_turns = int(round(self.Tx_turns * design.random_variable(variable_name=None, lower=0.6, upper=1.5, resolution=0.01), 0))

        preg = design.random_variable(lower=0.02, upper=0.06, resolution=0.001)


        self.Tx_preg = preg

        thick_ratio = design.random_variable(lower=0.25, upper=2.5, resolution=0.01)

        self.Tx_thick = 0.035 * thick_ratio

        self.Rx_preg = preg
        self.Rx_thick = 0.035 * thick_ratio


        self.Tx_Rx_gap = design.random_variable(lower=0.05, upper=0.25, resolution=0.01)

        self.Tx_shift = design.random_variable(lower=0, upper=0.5, resolution=0.01)
        self.Rx_shift = design.random_variable(lower=0, upper=0.5, resolution=0.01)

        self.Tx_shift = 0
        self.Rx_shift = 0

        # [
        #     self.Tx_turns, self.Rx_turns, self.Tx_outer_x, self.Tx_outer_y, self.Tx_ratio,
        #     self.Rx_outer_x, self.Rx_outer_y, self.Rx_ratio,
        #     self.Tx_inner, self.Rx_inner, self.Tx_fillet, self.Rx_fillet,
        #     self.Tx_fill_factor, self.Rx_fill_factor,
        #     self.Tx_preg, self.Rx_preg, self.Tx_thick, self.Rx_thick,
        #     self.Tx_Rx_gap, self.Tx_shift, self.Rx_shift
        # ] = [4.0, 7.0, 1.49, 1.34, 0.9, 1.45, 1.26, 0.87, 0.99, 0.56, 0.82, 0.46, 0.3, 0.3, 0.02, 0.02, 0.035/3, 0.035/3, 0.105, 0.0, 0.0]

        design["Tx_turns"] = f"{self.Tx_turns}"
        design["Rx_turns"] = f"{self.Rx_turns}"
        design["Tx_outer_x"] = f"{self.Tx_outer_x}mm"
        design["Tx_outer_y"] = f"{self.Tx_ratio} * {self.Tx_outer_x}mm"
        design["Rx_outer_x"] = f"{self.Rx_outer_x}mm"
        design["Rx_outer_y"] = f"{self.Rx_ratio} * {self.Rx_outer_x}mm"

        design["Tx_inner"] = f"{self.Tx_inner}mm"
        design["Rx_inner"] = f"{self.Rx_inner}mm"
        design["Tx_fillet"] = f"{self.Tx_fillet}mm"
        design["Rx_fillet"] = f"{self.Rx_fillet}mm"

        design["Tx_fill_factor"] = f"{self.Tx_fill_factor}"
        design["Rx_fill_factor"] = f"{self.Rx_fill_factor}"

        design["Tx_theta1"] = "15*pi/180"
        design["Tx_theta2"] = "75*pi/180"
        design["Rx_theta1"] = "15*pi/180"
        design["Rx_theta2"] = "75*pi/180"

        design["Tx_Rx_gap"] = f"{self.Tx_Rx_gap}mm"
        design["thick"] = f"{self.Tx_thick}mm"
        design["preg"] = f"{preg}mm"

        columns = [
            'Tx_turns', 'Rx_turns', 'Tx_outer_x', 'Tx_outer_y', 'Tx_ratio',
            'Rx_outer_x', 'Rx_outer_y', 'Rx_ratio', 'Tx_inner', 'Rx_inner',
            'Tx_fillet', 'Rx_fillet', 'Tx_fill_factor', 'Rx_fill_factor',
            'Tx_preg', 'Rx_preg', 'Tx_thick', 'Rx_thick', 'Tx_Rx_gap',
            'Tx_shift', 'Rx_shift'
        ]

        self.input_raw = [self.Tx_turns, self.Rx_turns, self.Tx_outer_x, self.Tx_outer_y, self.Tx_ratio, self.Rx_outer_x, self.Rx_outer_y, self.Rx_ratio, 
                      self.Tx_inner, self.Rx_inner, self.Tx_fillet, self.Rx_fillet, self.Tx_fill_factor, self.Rx_fill_factor,
                      self.Tx_preg, self.Rx_preg, thick_ratio, thick_ratio, self.Tx_Rx_gap, self.Tx_shift, self.Rx_shift]
        
        self.input = pd.DataFrame([self.input_raw], columns=columns)


    def create_model(self, design) :

        # Tx

        winding_variables = {"outer_x": "Tx_outer_x", "outer_y": "Tx_outer_y", "fillet": "Tx_fillet", "inner": "Tx_inner", "fill_factor": "Tx_fill_factor", "theta1": "Tx_theta1", "theta2": "Tx_theta2"}
        Tx_coil1 = design.model3d.winding.coil_points(turns=self.Tx_turns, **winding_variables)
        Tx_coil1.points.insert(0, [f"0mm", f"{Tx_coil1.points[0][1]}", Tx_coil1.points[0][2]])

        coil_Tx1 = design.model3d.winding.create_polyline(name="Tx_winding1", points=Tx_coil1.points, width=Tx_coil1.width, height=f"{self.Tx_thick}mm")
        self.coil = coil_Tx1

        # coil_Tx1_box = design.model3d.create_box(origin=Tx_coil1[0], )

        coil_Tx1.move([0,0,-(self.Tx_thick+self.Tx_preg)/2])

        # Tx_coil2 = copy.deepcopy(Tx_coil1)

        # Tx_coil2.points[-1] = [f"({Tx_coil2.points[-1][0]}) - {self.Tx_shift}*({Tx_coil2.wg})",f"({Tx_coil2.points[-1][1]})",f"{Tx_coil2.points[-1][2]}"]
        # Tx_coil2.points.append([f"({Tx_coil2.points[-1][0]})",f"({Tx_coil2.points[-1][1]}) + {self.Tx_shift}*({Tx_coil2.wg})",f"{Tx_coil2.points[-1][2]}"])
        # Tx_coil2.points[0] = [f"({Tx_coil2.points[0][0]})",f"({Tx_coil2.points[0][1]}) + {self.Tx_shift}*({Tx_coil2.wg})",f"{Tx_coil2.points[0][2]}"]

        # coil_Tx2 = design.model3d.winding.create_polyline(name="Tx_winding2", points=Tx_coil2.points, width=Tx_coil2.width, height=f"{self.Tx_thick}mm")
        # coil_Tx2.mirror(origin=[0,0,0], vector=[1,0,0])
        # coil_Tx2.move([f"-{self.Tx_shift}*({Tx_coil2.wg})",f"-{self.Tx_shift}*({Tx_coil2.wg})",-(self.Tx_thick+self.Tx_preg)/2])

        # coil_Tx = design.model3d.unite(assignment=[coil_Tx1, coil_Tx2, Tx_via1])
        # design.modeler.subtract(blank_list=coil_Tx, tool_list=Tx_via2, keep_originals=False)

        coil_Tx = coil_Tx1

        coil_Tx.transparency = 0
        coil_Tx.color = [255, 10, 10]
        coil_Tx.material_name = "copper"


        point1 = [f"{Tx_coil1.points[0][0]}-{Tx_coil1.width}/2", Tx_coil1.points[0][1], f"({Tx_coil1.points[0][2]} - ({self.Tx_thick})/2)mm"]
        point2 = [f"{Tx_coil1.points[0][0]}-{Tx_coil1.width}/2", Tx_coil1.points[0][1], f"({Tx_coil1.points[0][2]} + ({self.Tx_preg}) + 3*({self.Tx_thick})/2)mm"]
        Tx_ter1 = design.model3d.winding.create_polyline(name="Tx_ter1", points=[point1, point2], width=Tx_coil1.width, height=Tx_coil1.width)
        point1 = [f"{Tx_coil1.points[-1][0]}+{Tx_coil1.width}/2", Tx_coil1.points[-1][1], f"({Tx_coil1.points[-1][2]} - ({self.Tx_thick})/2)mm"]
        point2 = [f"{Tx_coil1.points[-1][0]}+{Tx_coil1.width}/2", Tx_coil1.points[-1][1], f"({Tx_coil1.points[-1][2]} + ({self.Tx_preg}) + 3*({self.Tx_thick})/2)mm"]
        Tx_ter2 = design.model3d.winding.create_polyline(name="Tx_ter2", points=[point1, point2], width=Tx_coil1.width, height=Tx_coil1.width)

        Tx_ter1.material_name = "pec"
        Tx_ter2.material_name = "pec"

        
        Tx_ter1.move([0,0,-(self.Tx_thick+self.Tx_preg)/2])
        Tx_ter2.move([0,0,-(self.Tx_thick+self.Tx_preg)/2])


        Tx_ter_edge1 = Tx_ter1.top_face_y.top_edge_z
        Tx_ter_edge2 = Tx_ter2.bottom_face_y.top_edge_z

        Tx_ter_edges = design.modeler.create_object_from_edge([Tx_ter_edge1, Tx_ter_edge2], non_model=False)
        Tx_ter_face = design.modeler.connect(Tx_ter_edges)

        Tx_port = design.lumped_port(assignment=Tx_ter_face[0], reference=[Tx_ter1], create_port_sheet=False, port_on_plane=True, integration_line=0, impedance=50, name="Tx_port", renormalize=True, deembed=False, terminals_rename=True)



        # Rx

        winding_variables = {"outer_x": "Rx_outer_x", "outer_y": "Rx_outer_y", "fillet": "Rx_fillet", "inner": "Rx_inner", "fill_factor": "Rx_fill_factor", "theta1": "Rx_theta1", "theta2": "Rx_theta2"}
        Rx_coil1 = design.model3d.winding.coil_points(turns=self.Rx_turns, **winding_variables)
        Rx_coil1.points.insert(0, [f"{Rx_coil1.points[0][0]}", f"{Rx_coil1.points[0][1]} - 1mm", Rx_coil1.points[0][2]])

        Rx_via1 = design.model3d.winding.create_via(name="Rx_via", center=Rx_coil1.points[-1], outer_R=f"0.8*{Rx_coil1.width}/2", inner_R=f"0.6*{Rx_coil1.width}/2", height=f"{2*self.Rx_thick+self.Rx_preg}mm", via_pad_R=f"1.2*{Rx_coil1.width}/2", via_pad_thick=f"{self.Rx_thick}mm")
        Rx_via2 = design.model3d.winding.create_via(name="Rx_sub_via", center=Rx_coil1.points[-1], outer_R=f"0.6*{Rx_coil1.width}/2", height="1mm*2")
        Rx_via1.move([0,0,-(self.Tx_Rx_gap+self.Tx_thick+self.Rx_thick+self.Tx_preg/2+self.Rx_preg/2)])
        Rx_via2.move([0,0,-(self.Tx_Rx_gap+self.Tx_thick+self.Rx_thick+self.Tx_preg/2+self.Rx_preg/2)])

        coil_Rx1 = design.model3d.winding.create_polyline(name="Rx_winding1", points=Rx_coil1.points, width=Rx_coil1.width, height=f"{self.Rx_thick}mm")
        coil_Rx1.move([0,0,(self.Rx_thick+self.Rx_preg)/2 - (self.Tx_Rx_gap+self.Tx_thick+self.Rx_thick+self.Tx_preg/2+self.Rx_preg/2)])

        Rx_coil2 = copy.deepcopy(Rx_coil1)

        Rx_coil2.points[-1] = [f"({Rx_coil2.points[-1][0]}) - {self.Rx_shift}*({Rx_coil2.wg})",f"({Rx_coil2.points[-1][1]})",f"{Rx_coil2.points[-1][2]}"]
        Rx_coil2.points.append([f"({Rx_coil2.points[-1][0]})",f"({Rx_coil2.points[-1][1]}) + {self.Rx_shift}*({Rx_coil2.wg})",f"{Rx_coil2.points[-1][2]}"])
        Rx_coil2.points[0] = [f"({Rx_coil2.points[0][0]})",f"({Rx_coil2.points[0][1]}) + {self.Rx_shift}*({Rx_coil2.wg})",f"{Rx_coil2.points[0][2]}"]

        coil_Rx2 = design.model3d.winding.create_polyline(name="Rx_winding2", points=Rx_coil2.points, width=Rx_coil2.width, height=f"{self.Rx_thick}mm")
        coil_Rx2.mirror(origin=[0,0,0], vector=[1,0,0])
        coil_Rx2.move([f"-{self.Rx_shift}*({Rx_coil2.wg})",f"-{self.Rx_shift}*({Rx_coil2.wg})",-(self.Rx_thick+self.Rx_preg)/2 - (self.Tx_Rx_gap+self.Tx_thick+self.Rx_thick+self.Tx_preg/2+self.Rx_preg/2)])

        coil_Rx = design.model3d.unite(assignment=[coil_Rx1, coil_Rx2, Rx_via1])
        design.modeler.subtract(blank_list=coil_Rx, tool_list=Rx_via2, keep_originals=False)

        coil_Rx.transparency = 0
        coil_Rx.color = [10, 10, 255]
        coil_Rx.material_name = "copper"


        point1 = Rx_coil1.points[0]
        point2 = [point1[0], f"({point1[1]})-0.05mm", point1[2]]
        Rx_ter1 = design.model3d.winding.create_polyline(name="Rx_ter1", points=[point1, point2], width=Rx_coil1.width, height=f"{self.Rx_thick}mm")
        Rx_ter2 = design.model3d.winding.create_polyline(name="Rx_ter2", points=[point1, point2], width=Rx_coil1.width, height=f"{self.Rx_thick}mm")

        Rx_ter1.material_name = "pec"
        Rx_ter2.material_name = "pec"

        Rx_ter1.mirror(origin=[0,0,0], vector=[1,0,0])
        Rx_ter1.move([f"-{self.Rx_shift}*({Rx_coil2.wg})",0,-(self.Rx_thick+self.Rx_preg)/2 - (self.Tx_Rx_gap+self.Tx_thick+self.Rx_thick+self.Tx_preg/2+self.Rx_preg/2)])
        Rx_ter2.move([0,0,(self.Rx_thick+self.Rx_preg)/2 - (self.Tx_Rx_gap+self.Tx_thick+self.Rx_thick+self.Tx_preg/2+self.Rx_preg/2)])


        Rx_ter_edge1 = Rx_ter1.bottom_face_y.top_edge_x
        Rx_ter_edge2 = Rx_ter2.bottom_face_y.bottom_edge_x

        Rx_ter_edges = design.modeler.create_object_from_edge([Rx_ter_edge1, Rx_ter_edge2], non_model=False)
        Rx_ter_face = design.modeler.connect(Rx_ter_edges)

        Rx_port = design.lumped_port(assignment=Rx_ter_face[0], reference=[Rx_ter1], create_port_sheet=False, port_on_plane=True, integration_line=0, impedance=50, name="Rx_port", renormalize=True, deembed=False, terminals_rename=True)




        # PCB

        if self.Tx_outer_x > self.Rx_outer_x :
            outer_x = "Tx_outer_x"
        else :
            outer_x = "Rx_outer_x"

        if self.Tx_outer_y > self.Rx_outer_y :
            outer_y = "Tx_outer_y"
        else :
            outer_y = "Rx_outer_y"   

        origin = [f"-(1.4*{outer_x})", f"-(1.4*{outer_y})", f"-{(self.Tx_preg/2+self.Tx_thick+self.Tx_Rx_gap+2*self.Rx_thick+self.Rx_preg)}mm"]
        dimension = [f"2*(1.4*{outer_x})", f"2*(1.4*{outer_y})", f"{(self.Tx_preg+2*self.Tx_thick+self.Tx_Rx_gap+2*self.Rx_thick+self.Rx_preg)}mm"]
        PCB = design.modeler.create_box(
            origin = origin,
            sizes = dimension,
            name = "PCB",
            material = "FR4_epoxy"
        )

        PCB.color = [0 ,128 ,64]
        PCB.transparency = 0

        design.modeler.subtract(blank_list=PCB, tool_list=[coil_Tx, coil_Rx, Tx_ter1, Tx_ter2, Rx_ter1, Rx_ter2], keep_originals=True)



        region = design.modeler.create_air_region(x_pos="175",x_neg="175", y_pos="175",y_neg="175", z_pos = "3500", z_neg="3500")
        design.assign_material(assignment=region, material="vacuum")
        design.assign_radiation_boundary_to_objects(assignment=region, name="Radiation")



    def analyze(self) :

        setup = self.design.create_setup("Setup1")
        setup.props["Frequency"] = "10MHz"
        # setup["MaximumPasses"] = 15
        # setup["MaxDeltaS"] = 0.03
        setup["MaximumPasses"] = 10
        setup["MaxDeltaS"] = 0.001


        oDesign = self.project.SetActiveDesign(self.design.name)
        oModule = oDesign.GetModule("AnalysisSetup")

        sweep1 = oModule.InsertFrequencySweep("Setup1", 
            [
                "NAME:Sweep",
                "IsEnabled:="		, True,
                "RangeType:="		, "LinearCount",
                "RangeStart:="		, "50kHz",
                "RangeEnd:="		, "1MHz",
                "RangeCount:="		, 100,
                [
                    "NAME:SweepRanges",
                    [
                        "NAME:Subrange",
                        "RangeType:="		, "LinearCount",
                        "RangeStart:="		, "1MHz",
                        "RangeEnd:="		, "10MHz",
                        "RangeCount:="		, 100
                    ],
                    [
                        "NAME:Subrange",
                        "RangeType:="		, "LinearCount",
                        "RangeStart:="		, "10MHz",
                        "RangeEnd:="		, "500MHz",
                        "RangeCount:="		, 400
                    ]
                ],
                "Type:="		, "Fast",
                "SaveFields:="		, True,
                "SaveRadFields:="	, False,
                "GenerateFieldsForAllFreqs:=", False
            ])


        current_dir = os.getcwd()
        folder_path = os.path.join(current_dir, "simulation", f"{self.PROJECT_NAME}")
        os.makedirs(folder_path, exist_ok=True)  # 폴더가 없으면 생성
        file_path = os.path.join(folder_path, f"{self.PROJECT_NAME}.aedt")

        self.project.save_project(path=file_path)


        # design1.analyze(setup=f"{setup.name} : {sweep1.name}", cores=4, tasks=1)
        self.design.analyze(setup=f"{setup.name}", cores=self.NUM_CORE, tasks=self.NUM_TASK)




        result_expressions = []
        result_expressions.append(f"mag(Zt(Tx_port_T1,Tx_port_T1))")
        result_expressions.append(f"mag(Zt(Rx_port_T1,Rx_port_T1))")
        result_expressions.append(f"mag(Zt(Tx_port_T1,Rx_port_T1))")

        report1 = self.design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                        variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                        report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="impedance_mag data")


        result_expressions = []
        result_expressions.append(f"re(Zt(Tx_port_T1,Tx_port_T1))")
        result_expressions.append(f"re(Zt(Rx_port_T1,Rx_port_T1))")
        result_expressions.append(f"re(Zt(Tx_port_T1,Rx_port_T1))")

        report2 = self.design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                        variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                        report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="impedance_real data")


        result_expressions = []
        result_expressions.append(f"im(Zt(Tx_port_T1,Tx_port_T1))")
        result_expressions.append(f"im(Zt(Rx_port_T1,Rx_port_T1))")
        result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))")
        result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))/sqrt(im(Zt(Tx_port_T1,Tx_port_T1))*im(Zt(Rx_port_T1,Rx_port_T1)))")

        report3 = self.design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                        variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                        report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="impedance_img data")



        result_expressions = []
        result_expressions.append(f"sqrt( im(Zt(Tx_port_T1,Tx_port_T1)) / im(Zt(Rx_port_T1,Rx_port_T1)) )")

        report4 = self.design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                        variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                        report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="voltage_ratio data")
        


        result_expressions = []
        result_expressions.append(f"im(Zt(Tx_port_T1,Tx_port_T1))/2/pi/freq")
        result_expressions.append(f"im(Zt(Rx_port_T1,Rx_port_T1))/2/pi/freq")
        result_expressions.append(f"im(Zt(Tx_port_T1,Rx_port_T1))/2/pi/freq")

        report5 = self.design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                        variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                        report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="inductance data")
        

        result_expressions = []
        result_expressions.append(f"dB(St(Tx_port_T1,Tx_port_T1))")
        result_expressions.append(f"dB(St(Rx_port_T1,Rx_port_T1))")
        result_expressions.append(f"dB(St(Tx_port_T1,Rx_port_T1))")

        report6 = self.design.post.create_report(expressions=result_expressions, setup_sweep_name=None, domain='Sweep', 
                                        variations=None, primary_sweep_variable=None, secondary_sweep_variable=None, 
                                        report_category=None, plot_type='Rectangular Plot', context=None, subdesign_id=None, polyline_points=1001, plot_name="return loss data")



        # project_dir = os.path.join(self.project.GetPath(), self.project.GetName())
        project_dir = self.project.GetPath()
        self.project_path = project_dir

        sim_data1 = self.design.post.export_report_to_csv(project_dir=project_dir, plot_name=report1.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
        sim_data2 = self.design.post.export_report_to_csv(project_dir=project_dir, plot_name=report2.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
        sim_data3 = self.design.post.export_report_to_csv(project_dir=project_dir, plot_name=report3.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
        sim_data4 = self.design.post.export_report_to_csv(project_dir=project_dir, plot_name=report4.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
        sim_data5 = self.design.post.export_report_to_csv(project_dir=project_dir, plot_name=report5.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)
        sim_data6 = self.design.post.export_report_to_csv(project_dir=project_dir, plot_name=report6.plot_name, uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

        data1 = pd.read_csv(sim_data1)
        data2 = pd.read_csv(sim_data2)
        data3 = pd.read_csv(sim_data3)
        data4 = pd.read_csv(sim_data4)
        data5 = pd.read_csv(sim_data5)
        data6 = pd.read_csv(sim_data6)

        self.data1 = self.design.post_processing.data_preprocessing(data1)
        self.data2 = self.design.post_processing.data_preprocessing(data2)
        self.data3 = self.design.post_processing.data_preprocessing(data3)
        self.data4 = self.design.post_processing.data_preprocessing(data4)
        self.data5 = self.design.post_processing.data_preprocessing(data5)
        self.data6 = self.design.post_processing.data_preprocessing(data6)



        
        data1_freq = data1["Freq [Hz]"]
        data1_ZTT = data1["mag(Zt(Tx_port_T1,Tx_port_T1)) []"]
        data1_ZRR = data1["mag(Zt(Rx_port_T1,Rx_port_T1)) []"]
        data1_ZTR = data1["mag(Zt(Tx_port_T1,Rx_port_T1)) []"]

        data2_freq = data2["Freq [Hz]"]
        data2_RTT = data2["re(Zt(Tx_port_T1,Tx_port_T1)) []"]
        data2_RRR = data2["re(Zt(Rx_port_T1,Rx_port_T1)) []"]
        data2_RTR = data2["re(Zt(Tx_port_T1,Rx_port_T1)) []"]

        data3_freq = data3["Freq [Hz]"]
        data3_ZTT = data3["im(Zt(Tx_port_T1,Tx_port_T1)) []"]
        data3_ZRR = data3["im(Zt(Rx_port_T1,Rx_port_T1)) []"]
        data3_ZTR = data3["im(Zt(Tx_port_T1,Rx_port_T1)) []"]
        data3_k = data3["im(Zt(Tx_port_T1,Rx_port_T1))/sqrt(im(Zt(Tx_port_T1,Tx_port_T1))*im(Zt(Rx_port_T1,Rx_port_T1))) []"]

        data4_freq = data4["Freq [Hz]"]
        data4_voltate_ratio = data4["sqrt( im(Zt(Tx_port_T1,Tx_port_T1)) / im(Zt(Rx_port_T1,Rx_port_T1)) ) []"]

        data5_freq = data5["Freq [Hz]"]
        data5_LTT = data5["im(Zt(Tx_port_T1,Tx_port_T1))/2/pi/freq []"]
        data5_LRR = data5["im(Zt(Rx_port_T1,Rx_port_T1))/2/pi/freq []"]
        data5_LTR = data5["im(Zt(Tx_port_T1,Rx_port_T1))/2/pi/freq []"]
        

        data6_freq = data6["Freq [Hz]"]
        data6_S11 = data6["dB(St(Tx_port_T1,Tx_port_T1)) []"]
        data6_S22 = data6["dB(St(Rx_port_T1,Rx_port_T1)) []"]
        data6_S12 = data6["dB(St(Tx_port_T1,Rx_port_T1)) []"]



        # get ratio data

        ratio_10k = self.design.post_processing.get_frequency_data(10e+3, data4_freq, data4_voltate_ratio) 
        ratio_100k = self.design.post_processing.get_frequency_data(100e+3, data4_freq, data4_voltate_ratio) 
        ratio_1M = self.design.post_processing.get_frequency_data(1e+6, data4_freq, data4_voltate_ratio) 
        ratio_10M = self.design.post_processing.get_frequency_data(10e+6, data4_freq, data4_voltate_ratio) 
        ratio_100M = self.design.post_processing.get_frequency_data(100e+6, data4_freq, data4_voltate_ratio) 

        columns = ['ratio_10k', 'ratio_100k', 'ratio_1M', 'ratio_10M', 'ratio_100M']
        self.voltage_ratio_raw = [ratio_10k, ratio_100k, ratio_1M, ratio_10M, ratio_100M]
        self.voltage_ratio = pd.DataFrame([self.voltage_ratio_raw], columns=columns)

        # get resistance data

        RT_10k = self.design.post_processing.get_frequency_data(10e+3, data2_freq, data2_RTT)
        RT_100k = self.design.post_processing.get_frequency_data(100e+3, data2_freq, data2_RTT)
        RT_1M = self.design.post_processing.get_frequency_data(1e+6, data2_freq, data2_RTT)
        RT_10M = self.design.post_processing.get_frequency_data(10e+6, data2_freq, data2_RTT)
        RT_100M = self.design.post_processing.get_frequency_data(100e+6, data2_freq, data2_RTT)

        RR_10k = self.design.post_processing.get_frequency_data(10e+3, data2_freq, data2_RRR)
        RR_100k = self.design.post_processing.get_frequency_data(100e+3, data2_freq, data2_RRR)
        RR_1M = self.design.post_processing.get_frequency_data(1e+6, data2_freq, data2_RTR)
        RR_10M = self.design.post_processing.get_frequency_data(10e+6, data2_freq, data2_RRR)
        RR_100M = self.design.post_processing.get_frequency_data(100e+6, data2_freq, data2_RRR)

        columns = ['RT_10k', 'RT_100k', 'RT_1M', 'RT_10M', 'RT_100M', 'RR_10k', 'RR_100k', 'RR_1M', 'RR_10M', 'RR_100M']
        self.R_raw = [RT_10k, RT_100k, RT_1M, RT_10M, RT_100M, RR_10k, RR_100k, RR_1M, RR_10M, RR_100M]
        self.R = pd.DataFrame([self.R_raw], columns=columns)

        # get inductance data

        LT_10k = self.design.post_processing.get_frequency_data(10e+3, data5_freq, data5_LTT) * 1e+6
        LT_100k = self.design.post_processing.get_frequency_data(100e+3, data5_freq, data5_LTT) * 1e+6
        LT_1M = self.design.post_processing.get_frequency_data(1e+6, data5_freq, data5_LTT) * 1e+6
        LT_10M = self.design.post_processing.get_frequency_data(10e+6, data5_freq, data5_LTT) * 1e+6
        LT_100M = self.design.post_processing.get_frequency_data(100e+6, data5_freq, data5_LTT) * 1e+6

        LR_10k = self.design.post_processing.get_frequency_data(10e+3, data5_freq, data5_LRR) * 1e+6
        LR_100k = self.design.post_processing.get_frequency_data(100e+3, data5_freq, data5_LRR) * 1e+6
        LR_1M = self.design.post_processing.get_frequency_data(1e+6, data5_freq, data5_LRR) * 1e+6
        LR_10M = self.design.post_processing.get_frequency_data(10e+6, data5_freq, data5_LRR) * 1e+6
        LR_100M = self.design.post_processing.get_frequency_data(100e+6, data5_freq, data5_LRR) * 1e+6

        LM_10k = self.design.post_processing.get_frequency_data(10e+3, data5_freq, data5_LTR) * 1e+6
        LM_100k = self.design.post_processing.get_frequency_data(100e+3, data5_freq, data5_LTR) * 1e+6
        LM_1M = self.design.post_processing.get_frequency_data(1e+6, data5_freq, data5_LTR) * 1e+6
        LM_10M = self.design.post_processing.get_frequency_data(10e+6, data5_freq, data5_LTR) * 1e+6
        LM_100M = self.design.post_processing.get_frequency_data(100e+6, data5_freq, data5_LTR) * 1e+6

        k_10k = abs(self.design.post_processing.get_frequency_data(10e+3, data5_freq, data3_k))
        k_100k = abs(self.design.post_processing.get_frequency_data(100e+3, data5_freq, data3_k))
        k_1M = abs(self.design.post_processing.get_frequency_data(1e+6, data5_freq, data3_k))
        k_10M = abs(self.design.post_processing.get_frequency_data(10e+6, data5_freq, data3_k))
        k_100M = abs(self.design.post_processing.get_frequency_data(100e+6, data5_freq, data3_k))

        columns = ['LT_10k', 'LT_100k', 'LT_1M', 'LT_10M', 'LT_100M', 'LR_10k', 'LR_100k', 'LR_1M', 'LR_10M', 'LR_100M', 
                   'LM_10k', 'LM_100k', 'LM_1M', 'LM_10M', 'LM_100M', 'k_10k', 'k_100k', 'k_1M', 'k_10M', 'k_100M']
        self.L_raw = [LT_10k, LT_100k, LT_1M, LT_10M, LT_100M, LR_10k, LR_100k, LR_1M, LR_10M, LR_100M, LM_10k, LM_100k, LM_1M, LM_10M, LM_100M, k_10k, k_100k, k_1M, k_10M, k_100M]
        self.L = pd.DataFrame([self.L_raw], columns=columns)

        # detect peak impedance

        freq1, peak1 = self.design.post_processing.detect_peak(freq=data1_freq, data=data1_ZTT)
        freq2, peak2 = self.design.post_processing.detect_peak(freq=data1_freq, data=data1_ZRR)
        freq3, peak3 = self.design.post_processing.detect_peak(freq=data1_freq, data=data1_ZTR)

        columns = ['TT_resonant_freq', 'TT_resonant_Z', 'RR_resonant_freq', 'RR_resonant_Z', 'TR_resonant_freq', 'TR_resonant_Z']
        self.resonant_raw = [freq1, peak1, freq2, peak2, freq3, peak3]
        self.resonant = pd.DataFrame([self.resonant_raw], columns=columns)

        # detect zero crossing
        freq_zero1 = self.design.post_processing.detect_zero_crossing(freq=data3_freq, data=data3_ZTT)
        freq_zero2 = self.design.post_processing.detect_zero_crossing(freq=data3_freq, data=data3_ZRR)
        freq_zero3 = self.design.post_processing.detect_zero_crossing(freq=data3_freq, data=data3_ZTR)

        columns = ['TT_zero_freq', 'RR_zero_freq', 'TR_zero_freq']
        self.zero_raw = [freq_zero1, freq_zero2, freq_zero3]
        self.zero = pd.DataFrame([self.zero_raw], columns=columns)

        # detect resonance

        self.freq1 = freq1
        self.freq2 = freq2
        self.freq3 = freq3

        self.peak1 = peak1
        self.peak2 = peak2
        self.peak3 = peak3

        self.freq_zero1 = freq_zero1
        self.freq_zero2 = freq_zero2
        self.freq_zero3 = freq_zero3

        freq1, peak1 = self.design.post_processing.detect_resonant(freq=freq1, peak=peak1, freq_zero=freq_zero1, tolerance=5)
        freq2, peak2 = self.design.post_processing.detect_resonant(freq=freq2, peak=peak2, freq_zero=freq_zero2, tolerance=5)
        freq3, peak3 = self.design.post_processing.detect_resonant(freq=freq3, peak=peak3, freq_zero=freq_zero3, tolerance=5)

        columns = ['TT_resonant_freq_final', 'RR_resonant_freq_final', 'TR_resonant_freq_final']
        self.resonant_final_raw = [[int(x) for x in freq1], [int(x) for x in freq2], [int(x) for x in freq3]]
        self.resonant_final = pd.DataFrame([self.resonant_final_raw], columns=columns)


        # detect S-parameter peak

        freq1, peak1 = self.detect_Speak(freq=data6_freq, data=data6_S11)
        freq2, peak2 = self.detect_Speak(freq=data6_freq, data=data6_S22)
        freq3, peak3 = self.detect_Speak(freq=data6_freq, data=data6_S12)

        columns = ['S11_peak_freq', 'S11_peak_value', 'S22_peak_freq', 'S22_peak_value', 'S12_peak_freq', 'S12_peak_value']
        self.S_final_raw = [[int(x) for x in freq1], [round(abs(float(x)), 2) for x in peak1], [int(x) for x in freq2], [round(abs(float(x)), 2) for x in peak2], [int(x) for x in freq3], [round(abs(float(x)), 2) for x in peak3]]
        self.S_final = pd.DataFrame([self.S_final_raw], columns=columns)



        # get S-parameter value

        S11_10M = -self.design.post_processing.get_frequency_data(10e+6, data6_freq, data6_S11)
        S11_15M = -self.design.post_processing.get_frequency_data(15e+6, data6_freq, data6_S11)
        S11_20M = -self.design.post_processing.get_frequency_data(20e+6, data6_freq, data6_S11)
        S11_25M = -self.design.post_processing.get_frequency_data(25e+6, data6_freq, data6_S11)
        S11_30M = -self.design.post_processing.get_frequency_data(30e+6, data6_freq, data6_S11)
        S11_40M = -self.design.post_processing.get_frequency_data(40e+6, data6_freq, data6_S11)
        S11_50M = -self.design.post_processing.get_frequency_data(50e+6, data6_freq, data6_S11)

        S22_10M = -self.design.post_processing.get_frequency_data(10e+6, data6_freq, data6_S22)
        S22_15M = -self.design.post_processing.get_frequency_data(15e+6, data6_freq, data6_S22)
        S22_20M = -self.design.post_processing.get_frequency_data(20e+6, data6_freq, data6_S22)
        S22_25M = -self.design.post_processing.get_frequency_data(25e+6, data6_freq, data6_S22)
        S22_30M = -self.design.post_processing.get_frequency_data(30e+6, data6_freq, data6_S22)
        S22_40M = -self.design.post_processing.get_frequency_data(40e+6, data6_freq, data6_S22)
        S22_50M = -self.design.post_processing.get_frequency_data(50e+6, data6_freq, data6_S22)

        S12_10M = -self.design.post_processing.get_frequency_data(10e+6, data6_freq, data6_S12)
        S12_15M = -self.design.post_processing.get_frequency_data(15e+6, data6_freq, data6_S12)
        S12_20M = -self.design.post_processing.get_frequency_data(20e+6, data6_freq, data6_S12)
        S12_25M = -self.design.post_processing.get_frequency_data(25e+6, data6_freq, data6_S12)
        S12_30M = -self.design.post_processing.get_frequency_data(30e+6, data6_freq, data6_S12)
        S12_40M = -self.design.post_processing.get_frequency_data(40e+6, data6_freq, data6_S12)
        S12_50M = -self.design.post_processing.get_frequency_data(50e+6, data6_freq, data6_S12)

        columns = ['S11_10M', 'S11_15M', 'S11_20M', 'S11_25M', 'S11_30M', 'S11_40M', 'S11_50M',
                   'S22_10M', 'S22_15M', 'S22_20M', 'S22_25M', 'S22_30M', 'S22_40M', 'S22_50M',
                   'S12_10M', 'S12_15M', 'S12_20M', 'S12_25M', 'S12_30M', 'S12_40M', 'S12_50M']
        self.S_raw = [S11_10M, S11_15M, S11_20M, S11_25M, S11_30M, S11_40M, S11_50M,
                      S22_10M, S22_15M, S22_20M, S22_25M, S22_30M, S22_40M, S22_50M,
                      S12_10M, S12_15M, S12_20M, S12_25M, S12_30M, S12_40M, S12_50M]
        self.S = pd.DataFrame([self.S_raw], columns=columns)






        # get report data

        pass_number, tetrahedra, delta_S = self.design.get_report_data(setup=setup.name)

        end_time = time.time()
        execution_time = int(end_time - self.start_time)

        columns = ['time', 'pass_number', 'tetrahedra', 'delta_S']
        self.sim_result_raw = [execution_time, pass_number, tetrahedra, delta_S]
        self.sim_result = pd.DataFrame([self.sim_result_raw], columns=columns)


        print("manual log : circuit start")

        self.circuit = self.project.create_design(name="circuit_design", solver="Circuit", solution=None)

        print("manual log : circuit complete")

        # ─── 3) COM ModelManager 에 넘길 파라미터 리스트 구성 ────────────────────────────

        image_file = os.path.join(os.getcwd(), "image.gif")

        self.project.CopyDesign(self.design.name)

        ModTime = int(time.time())

        proj         = self.circuit._oproject
        def_mgr      = proj.GetDefinitionManager()
        model_mgr    = def_mgr.GetManager("Model")
        params = [
            "NAME:Link_model",
            "Name:=",               "Link_model",
            "ModTime:=",            ModTime,
            "Library:=",            "",
            "LibLocation:=",        "Project",
            "ModelType:=",          "hfss",
            "Description:=",        "",
            "ImageFile:=",          image_file,
            "SymbolPinConfiguration:=", 0,
            ["NAME:PortInfoBlk"],
            ["NAME:PortOrderBlk"],
            "DesignName:=",         "HFSS_design",
            "SolutionName:=",       "Setup1 : Sweep",
            "NewToOldMap:=",        [],              # **필수**
            "OldToNewMap:=",        [],              # **필수**
            "PinNames:=",           ["Rx_port_T1", "Tx_port_T1"],
            ["NAME:DesignerCustomization",
                "DCOption:=",       0,
                "InterpOption:=",   0,
                "ExtrapOption:=",   1,
                "Convolution:=",    0,
                "Passivity:=",      0,
                "Reciprocal:=",     False,
                "ModelOption:=",    "",
                "DataType:=",       1
            ],
            ["NAME:NexximCustomization",
                "DCOption:=",       3,
                "InterpOption:=",   1,
                "ExtrapOption:=",   3,
                "Convolution:=",    0,
                "Passivity:=",      0,
                "Reciprocal:=",     False,
                "ModelOption:=",    "",
                "DataType:=",       2
            ],
            ["NAME:HSpiceCustomization",
                "DCOption:=",       1,
                "InterpOption:=",   2,
                "ExtrapOption:=",   3,
                "Convolution:=",    0,
                "Passivity:=",      0,
                "Reciprocal:=",     False,
                "ModelOption:=",    "",
                "DataType:=",       3
            ],
            "NoiseModelOption:=",    "External",
            "WB_SystemID:=",         "HFSS_design",
            "IsWBModel:=",           False,
            "filename:=",            "",
            "numberofports:=",       2,
            "Simulate:=",            False,
            "CloseProject:=",        False,
            "SaveProject:=",         True,
            "InterpY:=",             True,
            "InterpAlg:=",           "auto",
            "IgnoreDepVars:=",       False,
            "Renormalize:=",         False,
            "RenormImpedance:=",     50
        ]


        # ─── 4) HFSS 링크 모델 추가 및 저장 ────────────────────────────────────────────
        model_mgr.Add(params)







        # ====================================================
        comp_mgr = def_mgr.GetManager("Component")

        params_comp = [
            "NAME:Link_model",
            "Info:="   , [
                "Type:=",         8,
                "NumTerminals:=", 2,
                "DataSource:=",   "",
                "ModifiedOn:=",   ModTime,
                "Manufacturer:=", "",
                "Symbol:=",       "",      # VBScript 기록과 동일하게 빈 문자열
                "ModelNames:=",   "",      # 역시 빈 문자열
                "Footprint:=",    "",
                "Description:=",  "",
                "InfoTopic:=",    "",
                "InfoHelpFile:=", "",
                "IconFile:=",     "hfss.bmp",
                "Library:=",      "",
                "OriginalLocation:=", "Project",
                "IEEE:=",         "",
                "Author:=",       "",
                "OriginalAuthor:=", "",
                "CreationDate:=", ModTime,
                "ExampleFile:=",  "",
                "HiddenComponent:=", 0,
                "CircuitEnv:=",       0,
                "GroupID:=",          0
            ],
            "CircuitEnv:=", 0,
            "Refbase:=",    "S",
            "NumParts:=",   1,
            "ModSinceLib:=", False,
            # 터미널: VBScript 기록과 동일한 인덱스(0,1)
            "Terminal:=", ["Rx_port_T1","Rx_port_T1","A",False,0,1,"","Electrical","0"],
            "Terminal:=", ["Tx_port_T1","Tx_port_T1","A",False,1,1,"","Electrical","0"],
            ["NAME:Properties",
                "TextProp:=", ["Owner","RD","","HFSS"]
            ],
            "CompExtID:=", 5,
            # Parameters 블록: **모든** VariableProp + TextProp + MenuProp + ButtonProp
            ["NAME:Parameters",
                # -- 16개의 VariableProp (예시로 몇 개만 넣고, 나머지도 동일 패턴) --
                "VariableProp:=", ["Tx_turns","D","",f'{self.Tx_turns}'],
                "VariableProp:=", ["Rx_turns","D","",f'{self.Rx_turns}'],
                "VariableProp:=", ["Tx_outer_x","D","",f'{self.Tx_outer_x}mm'],
                "VariableProp:=", ["Tx_outer_y","HD","",f'{self.Tx_ratio}*{self.Tx_outer_x}mm'],
                "VariableProp:=", ["Rx_outer_x","D","",f'{self.Rx_outer_x}mm'],
                "VariableProp:=", ["Rx_outer_y","HD","",f'{self.Rx_ratio}*{self.Rx_outer_x}mm'],
                "VariableProp:=", ["Tx_inner","D","",f'{self.Tx_inner}mm'],
                "VariableProp:=", ["Rx_inner","D","",f'{self.Rx_inner}mm'],
                "VariableProp:=", ["Tx_fillet","D","",f'{self.Tx_fillet}mm'],
                "VariableProp:=", ["Rx_fillet","D","",f'{self.Rx_fillet}mm'],
                "VariableProp:=", ["Tx_fill_factor","D","",f'{self.Tx_fill_factor}'],
                "VariableProp:=", ["Rx_fill_factor","D","",f'{self.Rx_fill_factor}'],
                "VariableProp:=", ["Tx_theta1","HD","","15*pi/180"],
                "VariableProp:=", ["Tx_theta2","HD","","75*pi/180"],
                "VariableProp:=", ["Rx_theta1","HD","","15*pi/180"],
                "VariableProp:=", ["Rx_theta2","HD","","75*pi/180"],
                "TextProp:=",     ["ModelName","RD","","FieldSolver"],
                "MenuProp:=",     ["CoSimulator","SD","","Default",0],
                "ButtonProp:=",   ["CosimDefinition","SD","","Edit","Edit",40501,
                                    "ButtonPropClientData:=", []]
            ],
            # CosimDefinitions: recorded 값 그대로
            ["NAME:CosimDefinitions",
                ["NAME:CosimDefinition",
                    "CosimulatorType:=",       103,
                    "CosimDefName:=",          "Default",
                    "IsDefinition:=",          True,
                    "Connect:=",               True,
                    "ModelDefinitionName:="	, "Link_model",
                    "ShowRefPin2:="		, 2,
                    "LenPropName:="		, ""
                ],
                "DefaultCosim:=",            "Default"
            ]
        ]

        comp_mgr.Add(params_comp)


        oDesign = self.project.SetActiveDesign("circuit_design")
        oDesign.AddCompInstance("Link_model")

        print("manual log : link model")


        oModule = oDesign.GetModule("SimSetup")
        oModule.AddLinearNetworkAnalysis(
            [
                "NAME:SimSetup",
                "DataBlockID:="		, 16,
                "OptionName:="		, "(Default Options)",
                "AdditionalOptions:="	, "",
                "AlterBlockName:="	, "",
                "FilterText:="		, "",
                "AnalysisEnabled:="	, 1,
                "HasTDRComp:="		, 0,
                [
                    "NAME:OutputQuantities"
                ],
                [
                    "NAME:NoiseOutputQuantities"
                ],
                "Name:="		, "LinearFrequency",
                "LinearFrequencyData:="	, [False,0.1,False,"",False],
                [
                    "NAME:SweepDefinition",
                    "Variable:="		, "Freq",
                    "Data:="		, "10MHz",
                    "OffsetF1:="		, False,
                    "Synchronize:="		, 0
                ]
            ])

        oEditor = oDesign.SetActiveEditor("SchematicEditor")

        oEditor.CreateIPort(
            [
                "NAME:IPortProps",
                "Name:="		, "Port1",
                "Id:="			, 3
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1,
                "X:="			, 0.0127,
                "Y:="			, 0.0127,
                "Angle:="		, 0,
                "Flip:="		, False
            ])
        oDesign.UpdateSources(
            [
                "NAME:NexximSources",
                [
                    "NAME:NexximSources",
                    [
                        "NAME:Data"
                    ]
                ]
            ], 
            [
                "NAME:ComponentConfigurationData",
                [
                    "NAME:ComponentConfigurationData",
                    [
                        "NAME:EnabledPorts"
                    ],
                    [
                        "NAME:EnabledMultipleComponents"
                    ],
                    [
                        "NAME:EnabledAnalyses",
                        [
                            "NAME:Port1",
                            "Port1:="		, ["LinearFrequency"]
                        ]
                    ]
                ]
            ])
        oDesign.ChangePortProperty("Port1", 
            [
                "NAME:Port1",
                "IIPortName:="		, "Port1",
                "SymbolType:="		, 1,
                "DoPostProcess:="	, False
            ], 
            [
                [
                    "NAME:Properties",
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:rz",
                            "Value:="		, "50ohm"
                        ],
                        [
                            "NAME:iz",
                            "Value:="		, "0ohm"
                        ],
                        [
                            "NAME:pnum",
                            "Value:="		, "1"
                        ],
                        [
                            "NAME:EnableNoise",
                            "Value:="		, False
                        ],
                        [
                            "NAME:noisetemp",
                            "Value:="		, "16.85cel"
                        ]
                    ]
                ]
            ])

        oDesign.UpdateSources(
            [
                "NAME:NexximSources",
                [
                    "NAME:NexximSources",
                    [
                        "NAME:Data",
                        [
                            "NAME:VoltageSinusoidal1",
                            "DataId:="		, "Source0",
                            "Type:="		, 1,
                            "Output:="		, 0,
                            "NumPins:="		, 2,
                            "Netlist:="		, "V@ID %0 %1 *DC(DC=@DC) SIN(?VO(@VO) ?VA(@VA) ?FREQ(@FREQ) ?TD(@TD) ?ALPHA(@ALPHA) ?THETA(@THETA)) *TONE(TONE=@TONE) *ACMAG(AC @ACMAG @ACPHASE)",
                            "CompName:="		, "Nexxim Circuit Elements\\Independent Sources:V_SIN",
                            "FDSFileName:="		, "",
                            "BtnPropFileName:="	, "",
                            [
                                "NAME:Properties",
                                "TextProp:="		, ["LabelID","HD","Property string for netlist ID","V@ID"],
                                "ValueProp:="		, ["ACMAG","OD","AC magnitude for small-signal analysis (Volts)","3.1*sqrt(2) V",0],
                                "ValuePropNU:="		, ["ACPHASE","D","AC phase for small-signal analysis","0deg",0,"deg",							"AdditionalPropInfo:="	, ""],
                                "ValueProp:="		, ["DC","D","DC voltage (Volts)","0V",0],
                                "ValueProp:="		, ["VO","D","Voltage offset from zero (Volts)","0V",0],
                                "ValueProp:="		, ["VA","D","Voltage amplitude (Volts)","0V",0],
                                "ValueProp:="		, ["FREQ","OD","Frequency (Hz)","10MHz",0],
                                "ValueProp:="		, ["TD","D","Delay to start of sine wave (seconds)","0s",0],
                                "ValueProp:="		, ["ALPHA","D","Damping factor (1/seconds)","0",0],
                                "ValuePropNU:="		, ["THETA","D","Phase delay","0deg",0,"deg",							"AdditionalPropInfo:="	, ""],
                                "ValueProp:="		, ["TONE","D","Frequency (Hz) to use for harmonic balance analysis, should be a submultiple of (or equal to) the driving frequency and should also be included in the HB analysis setup","0Hz",0],
                                "TextProp:="		, ["ModelName","SHD","","V_SIN"],
                                "ButtonProp:="		, ["CosimDefinition","D","","Edit","Edit",40501,							"ButtonPropClientData:=", []],
                                "MenuProp:="		, ["CoSimulator","D","","DefaultNetlist",0]
                            ]
                        ]
                    ]
                ]
            ], 
            [
                "NAME:ComponentConfigurationData",
                [
                    "NAME:ComponentConfigurationData",
                    [
                        "NAME:EnabledPorts",
                        "VoltageSinusoidal1:="	, ["Port1"]
                    ],
                    [
                        "NAME:EnabledMultipleComponents",
                        "VoltageSinusoidal1:="	, []
                    ],
                    [
                        "NAME:EnabledAnalyses",
                        [
                            "NAME:Port1",
                            "Port1:="		, ["LinearFrequency"]
                        ],
                        [
                            "NAME:VoltageSinusoidal1",
                            "Port1:="		, ["LinearFrequency"]
                        ]
                    ]
                ]
            ])
        oDesign.ChangePortProperty("Port1", 
            [
                "NAME:Port1",
                "IIPortName:="		, "Port1",
                "SymbolType:="		, 1,
                "DoPostProcess:="	, False
            ], 
            [
                [
                    "NAME:Properties",
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:rz",
                            "Value:="		, "1e-06ohm"
                        ],
                        [
                            "NAME:iz",
                            "Value:="		, "0ohm"
                        ],
                        [
                            "NAME:pnum",
                            "Value:="		, "1"
                        ],
                        [
                            "NAME:EnableNoise",
                            "Value:="		, False
                        ],
                        [
                            "NAME:noisetemp",
                            "Value:="		, "16.85cel"
                        ]
                    ]
                ]
            ])
        oEditor.CreateComponent(
            [
                "NAME:ComponentProps",
                "Name:="		, "Nexxim Circuit Elements\\Probes:IPROBE",
                "Id:="			, "23"
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1,
                "X:="			, 0.01524,
                "Y:="			, 0.00254,
                "Angle:="		, 0,
                "Flip:="		, False
            ])
        oEditor.Move(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@IPROBE;23;12"]
            ], 
            [
                "NAME:MoveParameters",
                "xdelta:="		, -0.00254,
                "ydelta:="		, 0.00254,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.Rotate(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@IPROBE;23;12"]
            ], 
            [
                "NAME:RotateParameters",
                "Degrees:="		, 90,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.Rotate(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@IPROBE;23;12"]
            ], 
            [
                "NAME:RotateParameters",
                "Degrees:="		, 90,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.Rotate(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@IPROBE;23;12"]
            ], 
            [
                "NAME:RotateParameters",
                "Degrees:="		, 90,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@IPROBE;23;12:1"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:Name",
                            "Value:="		, "Iin"
                        ]
                    ]
                ]
            ])
        oEditor.CreateComponent(
            [
                "NAME:ComponentProps",
                "Name:="		, "Nexxim Circuit Elements\\Probes:IPROBE",
                "Id:="			, "24"
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1,
                "X:="			, -0.03302,
                "Y:="			, -0.00254,
                "Angle:="		, 0,
                "Flip:="		, False
            ])
        oEditor.Rotate(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@IPROBE;24;19"]
            ], 
            [
                "NAME:RotateParameters",
                "Degrees:="		, 90,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.Rotate(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@IPROBE;24;19"]
            ], 
            [
                "NAME:RotateParameters",
                "Degrees:="		, 90,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@IPROBE;24;19:1"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:Name",
                            "Value:="		, "Iout"
                        ]
                    ]
                ]
            ])
        oEditor.CreateComponent(
            [
                "NAME:ComponentProps",
                "Name:="		, "Nexxim Circuit Elements\\Resistors:RES_",
                "Id:="			, "25"
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1,
                "X:="			, -0.0508,
                "Y:="			, -0.00254,
                "Angle:="		, 0,
                "Flip:="		, False
            ])
        oEditor.CreateWire(
            [
                "NAME:WireData",
                "Name:="		, "",
                "Id:="			, 26,
                "Points:="		, ["(-0.045720, -0.002540)","(-0.038100, -0.002540)"]
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1
            ])
        oEditor.CreateWire(
            [
                "NAME:WireData",
                "Name:="		, "",
                "Id:="			, 32,
                "Points:="		, ["(-0.027940, -0.002540)","(-0.010160, -0.002540)"]
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1
            ])
        oEditor.CreateWire(
            [
                "NAME:WireData",
                "Name:="		, "",
                "Id:="			, 38,
                "Points:="		, ["(0.000000, -0.002540)","(0.012700, -0.002540)","(0.012700, 0.000000)"]
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1
            ])
        oEditor.CreateGround(
            [
                "NAME:GroundProps",
                "Id:="			, 44
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1,
                "X:="			, -0.05588,
                "Y:="			, -0.01778,
                "Angle:="		, 0,
                "Flip:="		, False
            ])
        oEditor.CreateWire(
            [
                "NAME:WireData",
                "Name:="		, "",
                "Id:="			, 48,
                "Points:="		, ["(-0.055880, -0.015240)","(-0.055880, -0.002540)"]
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1
            ])
        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@RES_;25;24:1"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:R",
                            "Value:="		, "28"
                        ]
                    ]
                ]
            ])
        oEditor.CreateWire(
            [
                "NAME:WireData",
                "Name:="		, "",
                "Id:="			, 53,
                "Points:="		, ["(0.012700, 0.010160)","(0.012700, 0.012700)"]
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1
            ])
        oEditor.CreateComponent(
            [
                "NAME:ComponentProps",
                "Name:="		, "Nexxim Circuit Elements\\Probes:VPROBE",
                "Id:="			, "26"
            ], 
            [
                "NAME:Attributes",
                "Page:="		, 1,
                "X:="			, 0.0254,
                "Y:="			, -0.00508,
                "Angle:="		, 0,
                "Flip:="		, False
            ])
        oEditor.Move(
            [
                "NAME:Selections",
                "Selections:="		, ["CompInst@VPROBE;26;57"]
            ], 
            [
                "NAME:MoveParameters",
                "xdelta:="		, -0.06604,
                "ydelta:="		, 0.00762,
                "Disconnect:="		, False,
                "Rubberband:="		, False
            ])
        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@VPROBE;26;57:1"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:Name",
                            "Value:="		, "Vout"
                        ]
                    ]
                ]
            ])
        oDesign.Analyze("LinearFrequency")
        oModule = oDesign.GetModule("OutputVariable")
        oModule.CreateOutputVariable("Pin", "0.5*re(V(Port1)*conjg(I(Iin)))", "LinearFrequency", "Standard", 
            [
                "NAME:Context",
                "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
            ])
        oModule.CreateOutputVariable("Pout", "0.5*re(V(Vout)*conjg(I(Iout)))", "LinearFrequency", "Standard", 
            [
                "NAME:Context",
                "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
            ])
        oModule.CreateOutputVariable("Gv", "mag(V(Vout))/mag(V(Port1))", "LinearFrequency", "Standard", 
            [
                "NAME:Context",
                "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
            ])
        oModule.CreateOutputVariable("Eff", "Pout/Pin*100", "LinearFrequency", "Standard", 
            [
                "NAME:Context",
                "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
            ])
        


        oModule = oDesign.GetModule("ReportSetup")
        oModule.CreateReport("Return Loss Table 1", "Standard", "Data Table", "LinearFrequency", 
            [
                "NAME:Context",
                "SimValueContext:="	, [3,0,2,0,False,False,-1,1,0,1,1,"",0,0]
            ], 
            [
                "Freq:="		, ["All"]
            ], 
            [
                "X Component:="		, "Freq",
                "Y Component:="		, ["Pin","Pout","Gv","Eff","mag(V(Port1))","mag(V(Vout))","mag(I(Iin))","mag(I(Iout))"]
            ])



        project_dir = self.project.GetPath()

        sim_data_RL = self.circuit.post.export_report_to_csv(project_dir=project_dir, plot_name="Return Loss Table 1", uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

        data_RL = pd.read_csv(sim_data_RL)
        self.data_RL = self.circuit.post_processing.data_preprocessing(data_RL)

        # 단위 변환 및 컬럼명 변경
        rename_dict = {}
        for col in data_RL.columns:
            if "[mV]" in col:
                data_RL[col] = data_RL[col] / 1000  # mV → V
                new_col = col.replace("[mV]", "[V]")
                rename_dict[col] = new_col
            elif "[mA]" in col:
                data_RL[col] = data_RL[col] / 1000  # mA → A
                new_col = col.replace("[mA]", "[A]")
                rename_dict[col] = new_col

        # 컬럼 이름 일괄 변경
        data_RL.rename(columns=rename_dict, inplace=True)

        data_Pin = round(data_RL["Pin []"].values[0],4)
        data_Pout = round(data_RL["Pout []"].values[0],4)
        data_Gv = round(data_RL["Gv []"].values[0],4)
        data_eff = round(data_RL["Eff []"].values[0],4)
        data_Vin = round(data_RL["mag(V(Port1)) [V]"].values[0],4)
        data_Vload = round(data_RL["mag(V(Vout)) [V]"].values[0],4)
        data_Iin = round(data_RL["mag(I(Iin)) [A]"].values[0],4)
        data_Iload = round(data_RL["mag(I(Iout)) [A]"].values[0],4)

        columns = ['Pin28', 'Pout28', 'Gv28', 'eff28', 'Vin28', 'Vload28', 'Iin28', 'Iload28']
        self.circuit_data_raw = [data_Pin, data_Pout, data_Gv, data_eff, data_Vin, data_Vload, data_Iin, data_Iload]
        self.circuit_data_28 = pd.DataFrame([self.circuit_data_raw], columns=columns)




        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@RES_;25;24"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:R",
                            "Value:="		, "50"
                        ]
                    ]
                ]
            ])
        oDesign.Analyze("LinearFrequency")

        sim_data_RL = self.circuit.post.export_report_to_csv(project_dir=project_dir, plot_name="Return Loss Table 1", uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

        data_RL = pd.read_csv(sim_data_RL)
        self.data_RL = self.circuit.post_processing.data_preprocessing(data_RL)

        # 단위 변환 및 컬럼명 변경
        rename_dict = {}
        for col in data_RL.columns:
            if "[mV]" in col:
                data_RL[col] = data_RL[col] / 1000  # mV → V
                new_col = col.replace("[mV]", "[V]")
                rename_dict[col] = new_col
            elif "[mA]" in col:
                data_RL[col] = data_RL[col] / 1000  # mA → A
                new_col = col.replace("[mA]", "[A]")
                rename_dict[col] = new_col

        # 컬럼 이름 일괄 변경
        data_RL.rename(columns=rename_dict, inplace=True)

        data_Pin = round(data_RL["Pin []"].values[0],4)
        data_Pout = round(data_RL["Pout []"].values[0],4)
        data_Gv = round(data_RL["Gv []"].values[0],4)
        data_eff = round(data_RL["Eff []"].values[0],4)
        data_Vin = round(data_RL["mag(V(Port1)) [V]"].values[0],4)
        data_Vload = round(data_RL["mag(V(Vout)) [V]"].values[0],4)
        data_Iin = round(data_RL["mag(I(Iin)) [A]"].values[0],4)
        data_Iload = round(data_RL["mag(I(Iout)) [A]"].values[0],4)

        columns = ['Pin50', 'Pout50', 'Gv50', 'eff50', 'Vin50', 'Vload50', 'Iin50', 'Iload50']
        self.circuit_data_raw = [data_Pin, data_Pout, data_Gv, data_eff, data_Vin, data_Vload, data_Iin, data_Iload]
        self.circuit_data_50 = pd.DataFrame([self.circuit_data_raw], columns=columns)



        
        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@RES_;25;24"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:R",
                            "Value:="		, "100"
                        ]
                    ]
                ]
            ])
        oDesign.Analyze("LinearFrequency")

        sim_data_RL = self.circuit.post.export_report_to_csv(project_dir=project_dir, plot_name="Return Loss Table 1", uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

        data_RL = pd.read_csv(sim_data_RL)
        self.data_RL = self.circuit.post_processing.data_preprocessing(data_RL)

        # 단위 변환 및 컬럼명 변경
        rename_dict = {}
        for col in data_RL.columns:
            if "[mV]" in col:
                data_RL[col] = data_RL[col] / 1000  # mV → V
                new_col = col.replace("[mV]", "[V]")
                rename_dict[col] = new_col
            elif "[mA]" in col:
                data_RL[col] = data_RL[col] / 1000  # mA → A
                new_col = col.replace("[mA]", "[A]")
                rename_dict[col] = new_col

        # 컬럼 이름 일괄 변경
        data_RL.rename(columns=rename_dict, inplace=True)

        data_Pin = round(data_RL["Pin []"].values[0],4)
        data_Pout = round(data_RL["Pout []"].values[0],4)
        data_Gv = round(data_RL["Gv []"].values[0],4)
        data_eff = round(data_RL["Eff []"].values[0],4)
        data_Vin = round(data_RL["mag(V(Port1)) [V]"].values[0],4)
        data_Vload = round(data_RL["mag(V(Vout)) [V]"].values[0],4)
        data_Iin = round(data_RL["mag(I(Iin)) [A]"].values[0],4)
        data_Iload = round(data_RL["mag(I(Iout)) [A]"].values[0],4)

        columns = ['Pin100', 'Pout100', 'Gv100', 'eff100', 'Vin100', 'Vload100', 'Iin100', 'Iload100']
        self.circuit_data_raw = [data_Pin, data_Pout, data_Gv, data_eff, data_Vin, data_Vload, data_Iin, data_Iload]
        self.circuit_data_100 = pd.DataFrame([self.circuit_data_raw], columns=columns)




        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@RES_;25;24"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:R",
                            "Value:="		, "200"
                        ]
                    ]
                ]
            ])
        oDesign.Analyze("LinearFrequency")

        sim_data_RL = self.circuit.post.export_report_to_csv(project_dir=project_dir, plot_name="Return Loss Table 1", uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

        data_RL = pd.read_csv(sim_data_RL)
        self.data_RL = self.circuit.post_processing.data_preprocessing(data_RL)

        # 단위 변환 및 컬럼명 변경
        rename_dict = {}
        for col in data_RL.columns:
            if "[mV]" in col:
                data_RL[col] = data_RL[col] / 1000  # mV → V
                new_col = col.replace("[mV]", "[V]")
                rename_dict[col] = new_col
            elif "[mA]" in col:
                data_RL[col] = data_RL[col] / 1000  # mA → A
                new_col = col.replace("[mA]", "[A]")
                rename_dict[col] = new_col

        # 컬럼 이름 일괄 변경
        data_RL.rename(columns=rename_dict, inplace=True)

        data_Pin = round(data_RL["Pin []"].values[0],4)
        data_Pout = round(data_RL["Pout []"].values[0],4)
        data_Gv = round(data_RL["Gv []"].values[0],4)
        data_eff = round(data_RL["Eff []"].values[0],4)
        data_Vin = round(data_RL["mag(V(Port1)) [V]"].values[0],4)
        data_Vload = round(data_RL["mag(V(Vout)) [V]"].values[0],4)
        data_Iin = round(data_RL["mag(I(Iin)) [A]"].values[0],4)
        data_Iload = round(data_RL["mag(I(Iout)) [A]"].values[0],4)

        columns = ['Pin200', 'Pout200', 'Gv200', 'eff200', 'Vin200', 'Vload200', 'Iin200', 'Iload200']
        self.circuit_data_raw = [data_Pin, data_Pout, data_Gv, data_eff, data_Vin, data_Vload, data_Iin, data_Iload]
        self.circuit_data_200 = pd.DataFrame([self.circuit_data_raw], columns=columns)



        oEditor.ChangeProperty(
            [
                "NAME:AllTabs",
                [
                    "NAME:PassedParameterTab",
                    [
                        "NAME:PropServers", 
                        "CompInst@RES_;25;24"
                    ],
                    [
                        "NAME:ChangedProps",
                        [
                            "NAME:R",
                            "Value:="		, "1000"
                        ]
                    ]
                ]
            ])
        oDesign.Analyze("LinearFrequency")

        sim_data_RL = self.circuit.post.export_report_to_csv(project_dir=project_dir, plot_name="Return Loss Table 1", uniform=False, start=None, end=None, step=None, use_trace_number_format=False)

        data_RL = pd.read_csv(sim_data_RL)
        self.data_RL = self.circuit.post_processing.data_preprocessing(data_RL)

        # 단위 변환 및 컬럼명 변경
        rename_dict = {}
        for col in data_RL.columns:
            if "[mV]" in col:
                data_RL[col] = data_RL[col] / 1000  # mV → V
                new_col = col.replace("[mV]", "[V]")
                rename_dict[col] = new_col
            elif "[mA]" in col:
                data_RL[col] = data_RL[col] / 1000  # mA → A
                new_col = col.replace("[mA]", "[A]")
                rename_dict[col] = new_col

        # 컬럼 이름 일괄 변경
        data_RL.rename(columns=rename_dict, inplace=True)

        data_Pin = round(data_RL["Pin []"].values[0],4)
        data_Pout = round(data_RL["Pout []"].values[0],4)
        data_Gv = round(data_RL["Gv []"].values[0],4)
        data_eff = round(data_RL["Eff []"].values[0],4)
        data_Vin = round(data_RL["mag(V(Port1)) [V]"].values[0],4)
        data_Vload = round(data_RL["mag(V(Vout)) [V]"].values[0],4)
        data_Iin = round(data_RL["mag(I(Iin)) [A]"].values[0],4)
        data_Iload = round(data_RL["mag(I(Iout)) [A]"].values[0],4)

        columns = ['Pin1000', 'Pout1000', 'Gv1000', 'eff1000', 'Vin1000', 'Vload1000', 'Iin1000', 'Iload1000']
        self.circuit_data_raw = [data_Pin, data_Pout, data_Gv, data_eff, data_Vin, data_Vload, data_Iin, data_Iload]
        self.circuit_data_1000 = pd.DataFrame([self.circuit_data_raw], columns=columns)



        




        # output data
        
        self.output_data = pd.concat([self.input, self.voltage_ratio, self.L, self.R, self.resonant_final, self.S_final, self.S, self.circuit_data_28, self.circuit_data_50, self.circuit_data_100, self.circuit_data_200, self.circuit_data_1000, self.sim_result], axis=1)



        current_dir = os.getcwd()
        csv_file = os.path.join(current_dir, f"output_data_circuit.csv")

        if os.path.isfile(csv_file):
            self.output_data.to_csv(csv_file, mode='a', index=False, header=False)
        else:
            self.output_data.to_csv(csv_file, mode='w', index=False, header=True)


        # self.design.delete_project(self.PROJECT_NAME)


def loging(msg):

    file_path = "log.txt"
    max_attempts = 5
    attempt = 0

    # 파일이 없으면 새로 생성하고, 있으면 append 모드로 엽니다.
    while attempt < max_attempts:
        try:
            with open(file_path, "a", encoding="utf-8") as file:
                file.write(msg + "\n")
            break  # 성공하면 루프 탈출
        except Exception as e:
            attempt += 1
            print(f"파일 쓰기 오류 발생: {e}. 재시도 {attempt}/{max_attempts}...")
            time.sleep(1)
    else:
        print("파일 쓰기에 계속 실패했습니다.")
        




def safe_open(filename, mode, retries=5, delay=1):
    """
    filename: 열 파일명
    mode: 열기 모드 (예: 'r', 'w', 'a')
    retries: 재시도 횟수
    delay: 재시도 전 대기 시간(초)
    """
    for i in range(retries):
        try:
            return open(filename, mode, newline='')
        except (IOError, OSError) as e:
            if i == retries - 1:
                raise e
            time.sleep(delay)


def log_simulation(number, state=None, pid=None, filename='log.csv'):
    """
    number: 기록할 숫자 값
    state: None이면 초기 기록, "fail"이면 Error, 그 외는 Finished로 업데이트
    pid: 기록할 프로세스 아이디 값 (인자로 받음)
    filename: 로그 파일명 (기본 'log.csv')

    파일이 없으면 헤더( Number, Status, StartTime, PID )와 함께 생성한 후,
    초기 호출 시 새로운 레코드를 추가하고, state가 전달되면 기존 레코드의 Status를 업데이트합니다.
    """
    lock_timeout = 10  # 락 타임아웃 시간(초)

    # 파일이 없으면 헤더를 포함하여 생성
    if not os.path.exists(filename):
        with portalocker.Lock(filename, 'w', timeout=lock_timeout, newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Number', 'Status', 'StartTime', 'PID', 'EndTime'])
    
    # 초기 기록인 경우: state가 None이면 해당 번호의 레코드가 있는지 확인 후 없으면 추가
    if state is None:
        exists = False
        with portalocker.Lock(filename, 'r', timeout=lock_timeout, newline='') as f:
            reader = csv.reader(f)
            for row in reader:
                if row and row[0] == str(number):
                    exists = True
                    break
        if not exists:
            start_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            with portalocker.Lock(filename, 'a', timeout=lock_timeout, newline='') as f:
                writer = csv.writer(f)
                writer.writerow([number, 'Simulation', start_time, pid, ""])
    else:
        # state가 전달된 경우: 기존 레코드의 상태 업데이트
        new_status = "Error" if state.lower() == "fail" else "Finished"
        end_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        with portalocker.Lock(filename, 'r+', timeout=lock_timeout, newline='') as f:
            # 파일의 모든 행을 읽고 리스트로 저장
            rows = list(csv.reader(f))
            updated_rows = []
            for row in rows:
                # 헤더나, 해당 번호의 상태가 "Simulation"인 경우만 업데이트
                if row and row[0] == str(number) and row[1] == "Simulation":
                    row[1] = new_status
                    row[4] = end_time
                updated_rows.append(row)
            # 파일 포인터를 맨 앞으로 돌리고 내용을 덮어씌운 후 파일 내용을 잘라냅니다.
            f.seek(0)
            writer = csv.writer(f)
            writer.writerows(updated_rows)
            f.truncate()




def save_error_log(project_name, error_info):
    error_folder = "error"
    if not os.path.exists(error_folder):
        os.makedirs(error_folder)
    error_file = os.path.join(error_folder, f"{project_name}_error.txt")
    with open(error_file, "w", encoding="utf-8") as f:
        f.write(error_info)



sim1 = Simulation()



for i in range(1):

    info_handler = logging.FileHandler("info.log")
    info_handler.setLevel(logging.DEBUG)

    try:
        # 시뮬레이션 코드 실행
        sim1.simulation()


        


        pid = sim1.desktop.pid
        # sim1.project.close()
        time.sleep(1)
        # sim1.project.delete_project_folder(path=sim1.project_path)

        end_time = time.time()
        execution_time = end_time - sim1.start_time

        log_simulation(number=sim1.num, state="finished")
        loging(f"{sim1.PROJECT_NAME} : simulation success!! {i} ({execution_time:.1f} sec)")

    except Exception as e:

        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        err_info = f"error : {sim1.PROJECT_NAME}\n"
        err_info = f"input : {sim1.input}\n"
        err_info += f"{str(e)}\n"
        err_info += traceback.format_exc()
        # 콘솔 stderr에 출력 및 플러시
        print(err_info, file=sys.stderr)
        
        sys.stderr.flush()
        # logging.error를 통해 simulation.log에도 기록
        logging.error(err_info, exc_info=True)
        # error 폴더에 에러 정보 저장
        save_error_log(sim1.PROJECT_NAME, err_info)
        # log.csv 업데이트 (fail 상태)
        log_simulation(number=sim1.num, state="fail")
        loging(f"{sim1.PROJECT_NAME} : {i} simulation Failed")
        sim1.desktop.kill_process()
        del sim1
        sim1 = Simulation()
        time.sleep(1)
